# 🏠 **HabiData: Limpieza y Preprocesamiento de Datos Inmobiliarios**
## Análisis de Precios de Propiedades en Antioquia, Colombia

---

### **📊 Contexto del Proyecto**
Este notebook presenta el proceso completo de **limpieza y preprocesamiento** de un dataset de propiedades inmobiliarias en Colombia, con enfoque específico en **Antioquia**. 

### **🎯 Objetivo**
Preparar un dataset de alta calidad para **predicción de precios** inmobiliarios, aplicando técnicas avanzadas de:
- ✅ Limpieza de datos
- ✅ Validación geográfica  
- ✅ Text mining de descripciones
- ✅ Imputación inteligente de valores faltantes

### **📝 Metodología**
Cada decisión de limpieza está **justificada técnicamente** y documentada para garantizar:
1. **Reproducibilidad** del proceso
2. **Transparencia** en las decisiones 
3. **Calidad científica** del análisis
4. **Optimización** para modelos predictivos

---

### **📋 Estructura del Análisis**
1. **Carga y Configuración Inicial**
2. **Filtrado Geográfico (Antioquia)**  
3. **Evaluación de Calidad de Datos**
4. **Limpieza de Precios Inválidos**
5. **Validación y Corrección Geográfica**
6. **Filtrado por Tipo de Propiedad**
7. **Análisis de Valores Faltantes**
8. **Text Mining de Descripciones**
9. **Estrategia de Integración de Datos**
10. **Análisis de Variables de Superficie**
11. **Tratamiento Final de Valores Faltantes**
12. **Validación y Resumen Final**

---

*Desarrollado como parte del análisis de mercado inmobiliario colombiano*

## 1️⃣ **Carga y Configuración Inicial**

### **🎯 Objetivo de Esta Sección**
Establecer el entorno de trabajo y cargar el dataset inicial de propiedades inmobiliarias colombianas.

### **📚 Justificación de Librerías**
- `pandas`: Manipulación eficiente de datasets grandes
- `numpy`: Operaciones numéricas optimizadas  
- `matplotlib/seaborn`: Visualización de patrones de datos
- `re`: Text mining de descripciones (procesamiento de lenguaje natural)
- `sqlalchemy`: Conexión a base de datos PostgreSQL (opcional)

In [25]:
# ===============================================================
# IMPORTACIÓN DE LIBRERÍAS ESENCIALES
# ===============================================================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import warnings

# Configuración para optimizar visualización
plt.style.use('default')
sns.set_palette("husl")
warnings.filterwarnings('ignore')

# Configuración de pandas para mejor visualización
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 100)

print("✅ Librerías importadas exitosamente")
print("✅ Configuración de visualización establecida")

✅ Librerías importadas exitosamente
✅ Configuración de visualización establecida


In [26]:
# ===============================================================
# CARGA DEL DATASET INICIAL
# ===============================================================

# Ruta al archivo CSV con datos de propiedades colombianas
csv_path = "/home/andresgonzalezrpo/bootcamp_data/project/habidata/data/co_properties.csv"

# Cargar dataset completo
df_original = pd.read_csv(csv_path)

# Información básica del dataset
print("🏠 DATASET CARGADO: Propiedades Inmobiliarias Colombia")
print("=" * 55)
print(f"📊 Dimensiones: {df_original.shape}")
print(f"📋 Registros: {df_original.shape[0]:,}")
print(f"📋 Variables: {df_original.shape[1]}")
print(f"💾 Tamaño en memoria: {df_original.memory_usage(deep=True).sum() / 1024**2:.1f} MB")

# Vista previa de primeras filas
print(f"\n📋 PRIMERAS 3 FILAS:")
df_original.head(3)

🏠 DATASET CARGADO: Propiedades Inmobiliarias Colombia
📊 Dimensiones: (1000000, 25)
📋 Registros: 1,000,000
📋 Variables: 25
💾 Tamaño en memoria: 1482.7 MB

📋 PRIMERAS 3 FILAS:


,id,ad_type,start_date,end_date,created_on,lat,lon,l1,l2,l3,l4,l5,l6,rooms,bedrooms,bathrooms,surface_total,surface_covered,price,currency,price_period,title,description,property_type,operation_type
0,KsjahK62rxcYKXXQjOdkqw==,Propiedad,2020-10-07,2021-10-09,2020-10-07,3.9210,-76.506000,Colombia,Valle del Cauca,NaN,NaN,NaN,NaN,NaN,6.0,7.0,NaN,NaN,1.300000e+09,COP,NaN,Casa Campestre en venta en darien 3469064,"HERMOSA CASA CAMPESTRE, &Aacute;REA 6,000 MT, UBICADA EN LA VIA BUGA - BUENAVENTURA, EN PARCELAC...",Casa,Venta
1,Y+gsBZYq1zu5NoR3V5oUGA==,Propiedad,2020-10-07,2021-01-06,2020-10-07,3.3577,-76.541811,Colombia,Valle del Cauca,Cali,Ciudad Jardín,NaN,NaN,NaN,NaN,7.0,NaN,NaN,2.800000e+09,COP,NaN,Casa en ciudsd jardin,Casa independiente con posiciona en ciudad jardín y hermosos jardines. La casa s eve de amoblada...,Casa,Venta
2,Jpzqxj8/Vgf3Aa5ASxUBNg==,Propiedad,2020-10-07,2020-10-07,2020-10-07,3.3577,-76.541811,Colombia,Valle del Cauca,Cali,Ciudad Jardín,NaN,NaN,NaN,NaN,7.0,NaN,NaN,2.800000e+09,COP,Mensual,Casa en ciudsd jardin,Casa independiente con posiciona en ciudad jardín y hermosos jardines. Amo la casa al\nSur se la...,Casa,Venta


## 2️⃣ **Filtrado Geográfico: Enfoque en Antioquia**

### **🎯 Decisión Estratégica**
**¿Por qué filtrar solo Antioquia?**

1. **🏙️ Homogeneidad del Mercado**: Antioquia tiene un mercado inmobiliario más homogéneo con Medellín como centro económico
2. **📊 Volumen Suficiente**: Concentra gran cantidad de transacciones para análisis estadísticamente significativo
3. **🎯 Especificidad Regional**: Los precios inmobiliarios varían significativamente entre departamentos
4. **🔍 Calidad del Análisis**: Enfoque regional permite mayor precisión en patrones de precios

### **✅ Impacto Esperado**
- Reducir variabilidad geográfica extrema
- Mejorar homogeneidad para modelos predictivos  
- Mantener volumen suficiente para análisis robusto

In [27]:
# ===============================================================
# FILTRADO GEOGRÁFICO: ANTIOQUIA CON ANÁLISIS COMPLETO
# ===============================================================

# 1. Verificar distribución por departamento (l2)
print("🌍 DISTRIBUCIÓN POR DEPARTAMENTO (TOP 10):")
print("-" * 45)
departamentos = df_original['l2'].value_counts().head(10)
total_nacional = len(df_original)

for dept, count in departamentos.items():
    porcentaje = (count / total_nacional) * 100
    print(f"   {dept:<20}: {count:>8,} ({porcentaje:>5.1f}%)")

# 2. Filtrar únicamente Antioquia  
df_antioquia = df_original[df_original['l2'] == 'Antioquia'].copy()

# 3. Resumen del filtrado geográfico
print(f"\n✅ RESULTADO DEL FILTRADO GEOGRÁFICO:")
print("=" * 45)
print(f"📊 Dataset original: {len(df_original):,} propiedades")
print(f"📊 Dataset Antioquia: {len(df_antioquia):,} propiedades") 
print(f"📈 Porcentaje conservado: {(len(df_antioquia)/len(df_original)*100):.1f}%")
print(f"📍 Enfoque regional: Departamento de Antioquia únicamente")

# 4. Análisis de ciudades en Antioquia
print(f"\n🏙️ PRINCIPALES CIUDADES EN ANTIOQUIA:")
print("-" * 40)
ciudades_antioquia = df_antioquia['l3'].value_counts().head(8)
for ciudad, count in ciudades_antioquia.items():
    porcentaje = (count / len(df_antioquia)) * 100
    print(f"   {ciudad:<20}: {count:>6,} ({porcentaje:>5.1f}%)")

# 5. Lista de barrios/sectores (l4)
print(f"\n🏘️ BARRIOS/SECTORES EN ANTIOQUIA:")
print("-" * 40)

# Limpieza básica y conteo de barrios
conteo_barrios = (
    df_antioquia['l4']
    .astype(str)
    .str.strip()
    .str.title()
    .replace('Nan', np.nan)
    .value_counts()
)

# Mostrar listado de barrios
print(f"📋 LISTADO DE BARRIOS (ORDENADO POR CANTIDAD):")
print(f"{'#':<4} {'Barrio/Sector':<35} {'Propiedades':<12} {'%':<6}")
print("-" * 65)

for i, (barrio, count) in enumerate(conteo_barrios.items(), 1):
    porcentaje = (count / len(df_antioquia)) * 100
    print(f"{i:>3}. {str(barrio):<35}: {count:>8,} ({porcentaje:>4.1f}%)")

print(f"\n✅ TOTAL: {len(conteo_barrios):,} barrios únicos")

🌍 DISTRIBUCIÓN POR DEPARTAMENTO (TOP 10):
---------------------------------------------
   Antioquia           :  341,453 ( 34.1%)
   Cundinamarca        :  208,918 ( 20.9%)
   Valle del Cauca     :  117,770 ( 11.8%)
   Atlántico           :   78,605 (  7.9%)
   Santander           :   71,737 (  7.2%)
   Caldas              :   56,296 (  5.6%)
   Norte de Santander  :   32,247 (  3.2%)
   Risaralda           :   28,505 (  2.9%)
   Quindío             :   11,753 (  1.2%)
   Bolívar             :   11,016 (  1.1%)

✅ RESULTADO DEL FILTRADO GEOGRÁFICO:
📊 Dataset original: 1,000,000 propiedades
📊 Dataset Antioquia: 341,453 propiedades
📈 Porcentaje conservado: 34.1%
📍 Enfoque regional: Departamento de Antioquia únicamente

🏙️ PRINCIPALES CIUDADES EN ANTIOQUIA:
----------------------------------------
   Medellín            : 262,856 ( 77.0%)
   Envigado            : 24,171 (  7.1%)
   Sabaneta            : 10,836 (  3.2%)
   Bello               :  8,728 (  2.6%)
   Rionegro            :  8,

## 3️⃣ **Evaluación de Calidad de Datos**

### **🎯 Objetivo**
Identificar problemas de calidad que afecten el modelo de predicción de precios.

### **🔍 Estrategia**
1. **Valores faltantes** por variable
2. **Registros duplicados** completos
3. **Rangos lógicos** en variables numéricas
4. **Consistencia** en variables categóricas

In [28]:
# ===============================================================
# EVALUACIÓN RÁPIDA DE CALIDAD DE DATOS
# ===============================================================

print("🔍 EVALUACIÓN DE CALIDAD - DATASET ANTIOQUIA")
print("=" * 50)

# 1. Información básica
print(f"📊 Dimensiones: {df_antioquia.shape}")
print(f"📊 Tipos de datos:")
print(df_antioquia.dtypes.value_counts())

# 2. Valores faltantes críticos
print(f"\n🚨 VALORES FALTANTES (Top 10):")
missing = df_antioquia.isnull().sum()
missing_pct = (missing / len(df_antioquia)) * 100
missing_top = missing[missing > 0].sort_values(ascending=False).head(10)

for col, count in missing_top.items():
    pct = missing_pct[col]
    print(f"   {col:<20}: {count:>8,} ({pct:>5.1f}%)")

# 3. Duplicados
duplicados = df_antioquia.duplicated().sum()
print(f"\n📋 REGISTROS DUPLICADOS: {duplicados:,}")

# 4. Problemas en variables clave
print(f"\n⚠️ PROBLEMAS DETECTADOS:")

# Precios inválidos
precios_invalidos = (df_antioquia['price'] <= 0).sum()
print(f"   • Precios ≤ 0: {precios_invalidos:,}")

# Coordenadas fuera de Colombia
LAT_MIN, LAT_MAX = -4.23, 15.52
LON_MIN, LON_MAX = -79.01, -66.85
coords_invalidas = (
    (df_antioquia['lat'] < LAT_MIN) | (df_antioquia['lat'] > LAT_MAX) |
    (df_antioquia['lon'] < LON_MIN) | (df_antioquia['lon'] > LON_MAX)
).sum()
print(f"   • Coordenadas fuera de Colombia: {coords_invalidas:,}")

# Habitaciones extremas
if 'bedrooms' in df_antioquia.columns:
    bedrooms_extremas = (df_antioquia['bedrooms'] > 15).sum()
    print(f"   • Dormitorios > 15: {bedrooms_extremas:,}")

print(f"\n✅ EVALUACIÓN COMPLETADA")

🔍 EVALUACIÓN DE CALIDAD - DATASET ANTIOQUIA
📊 Dimensiones: (341453, 25)
📊 Tipos de datos:
object     17
float64     8
Name: count, dtype: int64

🚨 VALORES FALTANTES (Top 10):
   l6                  :  341,453 (100.0%)
   l5                  :  341,453 (100.0%)
   surface_total       :  334,859 ( 98.1%)
   surface_covered     :  334,412 ( 97.9%)
   price_period        :  314,524 ( 92.1%)
   rooms               :  305,597 ( 89.5%)
   l4                  :  262,728 ( 76.9%)
   bedrooms            :  261,106 ( 76.5%)
   lat                 :  196,750 ( 57.6%)
   lon                 :  196,750 ( 57.6%)

📋 REGISTROS DUPLICADOS: 0

⚠️ PROBLEMAS DETECTADOS:
   • Precios ≤ 0: 7
   • Coordenadas fuera de Colombia: 29
   • Dormitorios > 15: 54

✅ EVALUACIÓN COMPLETADA


## 4️⃣ **Limpieza de Precios Inválidos**

### **🎯 Decisión**
Eliminar registros con precio ≤ 0.

### **✅ Justificación**
Precios inválidos no sirven para predicción de precios.

In [64]:
# ===============================================================
# ELIMINAR PRECIOS INVÁLIDOS Y FILTRAR POR MONEDA
# ===============================================================

print("💰 ANÁLISIS DE MONEDA Y LIMPIEZA DE PRECIOS")
print("=" * 50)

inicial = len(df_antioquia)
print(f"📊 Dataset inicial: {inicial:,} registros")

# PASO 1: Verificar existencia y contenido de la columna currency
print(f"\n🔍 PASO 1: ANÁLISIS DE LA COLUMNA 'currency'")
print("-" * 45)

if 'currency' in df_antioquia.columns:
    print("✅ Columna 'currency' encontrada")
    
    # Verificar valores faltantes
    currency_nulos = df_antioquia['currency'].isna().sum()
    currency_completitud = ((len(df_antioquia) - currency_nulos) / len(df_antioquia)) * 100
    
    print(f"📊 Datos faltantes: {currency_nulos:,} ({100-currency_completitud:.1f}%)")
    print(f"📊 Datos completos: {len(df_antioquia) - currency_nulos:,} ({currency_completitud:.1f}%)")
    
    # Mostrar distribución de monedas
    print(f"\n📋 DISTRIBUCIÓN DE MONEDAS:")
    currency_dist = df_antioquia['currency'].value_counts(dropna=False)
    for moneda, count in currency_dist.items():
        porcentaje = (count / len(df_antioquia)) * 100
        estado = "✅" if str(moneda) == "COP" else "❌" if pd.notna(moneda) else "⚠️"
        print(f"   {estado} {moneda}: {count:,} ({porcentaje:.1f}%)")
    
    # PASO 2: Filtrar solo COP
    print(f"\n🎯 PASO 2: FILTRO DE MONEDA - SOLO PESOS COLOMBIANOS")
    print("-" * 55)
    
    antes_currency = len(df_antioquia)
    
    # Filtrar solo registros con currency = 'COP'
    df_cop = df_antioquia[df_antioquia['currency'] == 'COP'].copy()
    
    despues_currency = len(df_cop)
    eliminados_currency = antes_currency - despues_currency
    
    print(f"📊 Antes del filtro: {antes_currency:,}")
    print(f"✅ Después del filtro: {despues_currency:,}")
    print(f"🗑️ Eliminados (otras monedas): {eliminados_currency:,}")
    print(f"📈 Conservado: {(despues_currency/antes_currency*100):.1f}%")
    
    # Usar el dataset filtrado por moneda
    df_para_precio = df_cop.copy()
    
else:
    print("❌ Columna 'currency' NO encontrada")
    print("   → Asumiendo que todos los precios están en pesos colombianos")
    df_para_precio = df_antioquia.copy()

# PASO 3: Eliminar precios inválidos (≤ 0)
print(f"\n🏷️ PASO 3: ELIMINAR PRECIOS INVÁLIDOS")
print("-" * 40)

antes_precio = len(df_para_precio)
print(f"📊 Antes: {antes_precio:,}")

# Eliminar precios ≤ 0
df_clean = df_para_precio[df_para_precio['price'] > 0].copy()

despues_precio = len(df_clean)
eliminados_precio = antes_precio - despues_precio

print(f"✅ Después: {despues_precio:,}")
print(f"🗑️ Eliminados (precios ≤ 0): {eliminados_precio:,}")
print(f"📈 Conservado: {(despues_precio/antes_precio*100):.1f}%")

# RESUMEN FINAL
print(f"\n✅ RESUMEN FINAL DE LIMPIEZA:")
print("=" * 35)
print(f"📊 Dataset inicial: {inicial:,}")
print(f"📊 Dataset final: {len(df_clean):,}")
eliminados_total = inicial - len(df_clean)
print(f"🗑️ Total eliminados: {eliminados_total:,}")
print(f"📈 Conservación total: {(len(df_clean)/inicial*100):.1f}%")
print(f"💰 Moneda confirmada: Pesos Colombianos (COP)")

💰 ANÁLISIS DE MONEDA Y LIMPIEZA DE PRECIOS
📊 Dataset inicial: 341,453 registros

🔍 PASO 1: ANÁLISIS DE LA COLUMNA 'currency'
---------------------------------------------
✅ Columna 'currency' encontrada
📊 Datos faltantes: 79 (0.0%)
📊 Datos completos: 341,374 (100.0%)

📋 DISTRIBUCIÓN DE MONEDAS:
   ✅ COP: 341,366 (100.0%)
   ⚠️ nan: 79 (0.0%)
   ❌ USD: 7 (0.0%)
   ❌ ARS: 1 (0.0%)

🎯 PASO 2: FILTRO DE MONEDA - SOLO PESOS COLOMBIANOS
-------------------------------------------------------
📊 Antes del filtro: 341,453
✅ Después del filtro: 341,366
🗑️ Eliminados (otras monedas): 87
📈 Conservado: 100.0%

🏷️ PASO 3: ELIMINAR PRECIOS INVÁLIDOS
----------------------------------------
📊 Antes: 341,366
✅ Después: 341,365
🗑️ Eliminados (precios ≤ 0): 1
📈 Conservado: 100.0%

✅ RESUMEN FINAL DE LIMPIEZA:
📊 Dataset inicial: 341,453
📊 Dataset final: 341,365
🗑️ Total eliminados: 88
📈 Conservación total: 100.0%
💰 Moneda confirmada: Pesos Colombianos (COP)
✅ Después: 341,365
🗑️ Eliminados (precios ≤ 0): 

## 5️⃣ **Eliminar Coordenadas Inválidas**

### **🎯 Decisión**
Eliminar registros con coordenadas fuera de Colombia.

### **✅ Justificación**
Variables geográficas son críticas para predicción de precios.

In [30]:
# ===============================================================
# ELIMINAR COORDENADAS FUERA DE COLOMBIA
# ===============================================================

print("🌍 LIMPIEZA GEOGRÁFICA")
print("-" * 25)

# Rangos Colombia
LAT_MIN, LAT_MAX = -4.23, 15.52
LON_MIN, LON_MAX = -79.01, -66.85

inicial = len(df_clean)
print(f"📊 Antes: {inicial:,}")

# Identificar coordenadas fuera de Colombia
coords_invalidas = (
    (df_clean['lat'].notna()) & (df_clean['lon'].notna()) &
    ((df_clean['lat'] < LAT_MIN) | (df_clean['lat'] > LAT_MAX) |
     (df_clean['lon'] < LON_MIN) | (df_clean['lon'] > LON_MAX))
)

invalidas_count = coords_invalidas.sum()
print(f"🚨 Coordenadas fuera Colombia: {invalidas_count:,}")

# Eliminar registros inválidos
if invalidas_count > 0:
    df_clean = df_clean[~coords_invalidas].copy()

final = len(df_clean)
eliminados = inicial - final

print(f"✅ Después: {final:,}")
print(f"🗑️ Eliminados: {eliminados:,}")
print(f"📈 Conservado: {(final/inicial*100):.1f}%")

🌍 LIMPIEZA GEOGRÁFICA
-------------------------
📊 Antes: 341,373
🚨 Coordenadas fuera Colombia: 29
✅ Después: 341,344
🗑️ Eliminados: 29
📈 Conservado: 100.0%


## 6️⃣ **Filtros por Tipo de Propiedad**

**Decisión:** Enfocarse en apartamentos y casas únicamente

**Justificación:** Los lotes, locales comerciales y fincas tienen dinámicas de precio completamente diferentes. Concentrarse en vivienda residencial garantiza homogeneidad en el análisis

In [31]:
# ===============================================================
# FILTRAR TIPOS DE PROPIEDAD RESIDENCIAL
# ===============================================================

print("🏠 TIPOS DE PROPIEDAD")
print("-" * 25)

inicial = len(df_clean)
print(f"📊 Antes: {inicial:,}")

# Identificar tipos únicos
tipos_unicos = df_clean['property_type'].value_counts()
print(f"\n📝 Tipos de propiedad:")
for tipo, count in tipos_unicos.head(10).items():
    print(f"   {tipo}: {count:,}")

# Filtrar solo vivienda residencial
tipos_residenciales = ['Casa', 'Apartamento', 'casa', 'apartamento']
df_clean = df_clean[df_clean['property_type'].isin(tipos_residenciales)].copy()

final = len(df_clean)
eliminados = inicial - final

print(f"\n✅ Después filtro: {final:,}")
print(f"🗑️ Eliminados: {eliminados:,}")
print(f"📈 Conservado: {(final/inicial*100):.1f}%")

🏠 TIPOS DE PROPIEDAD
-------------------------
📊 Antes: 341,344

📝 Tipos de propiedad:
   Apartamento: 236,330
   Casa: 41,526
   Otro: 38,439
   Lote: 15,352
   Local comercial: 4,688
   Oficina: 3,624
   Finca: 1,142
   Depósito: 218
   Parqueadero: 25

✅ Después filtro: 277,856
🗑️ Eliminados: 63,488
📈 Conservado: 81.4%


## 7️⃣ **Extracción por Text Mining** 💎

**Decisión:** Extraer información de superficie desde descripciones usando expresiones regulares

**Justificación:** Muchas propiedades tienen superficie en el campo `description` pero no en `surface_total`. Esta es una **innovación clave** que recupera datos valiosos perdidos

In [32]:
# ===============================================================
# EXTRACCIÓN TEXT MINING - FUNCIÓN OPTIMIZADA
# ===============================================================

print("⛏️ TEXT MINING - SUPERFICIE OPTIMIZADA")
print("-" * 40)

def extraer_superficie_optimizada(descripcion):
    """Función optimizada para extraer superficie con múltiples patrones"""
    if pd.isna(descripcion):
        return None
    
    desc_lower = str(descripcion).lower()
    
    # Lista de patrones ordenados por especificidad
    patrones = [
        # Patrones principales (más específicos)
        r'(\d+(?:[.,]\d+)?)\s*(?:m2|m²|metros\s*cuadrados)',
        r'(\d+(?:[.,]\d+)?)\s*(?:mts2|mt2|metros2)',
        r'(\d+(?:[.,]\d+)?)\s*(?:metros|mts|metro)\s*(?:cuadrados?|construidos?)',
        
        # Patrones con contexto
        r'área\s*(?:de\s*|total\s*|construida\s*)?(\d+(?:[.,]\d+)?)',
        r'superficie\s*(?:de\s*|total\s*)?(\d+(?:[.,]\d+)?)',
        r'construidos?\s*(\d+(?:[.,]\d+)?)',
        r'(\d+(?:[.,]\d+)?)\s*(?:metros\s*construidos?)',
        
        # Patrones menos específicos
        r'(\d+(?:[.,]\d+)?)\s*m\s*(?:cuadrados?|construidos?)',
    ]
    
    for patron in patrones:
        matches = re.findall(patron, desc_lower)
        if matches:
            superficie_str = matches[0].replace(',', '.')
            try:
                valor = float(superficie_str)
                if 15 <= valor <= 2000:  # Rango razonable
                    return valor
            except:
                continue
    return None

# Análisis inicial
sin_superficie = df_clean['surface_total'].isna().sum()
print(f"🔍 Registros sin superficie: {sin_superficie:,}")

# Aplicar extracción optimizada
mask_sin_superficie = df_clean['surface_total'].isna()

# Limpiar columna anterior si existe
if 'surface_extracted' in df_clean.columns:
    df_clean = df_clean.drop('surface_extracted', axis=1)

# Crear nueva columna con extracciones
extracciones = df_clean.loc[mask_sin_superficie, 'description'].apply(extraer_superficie_optimizada)
df_clean.loc[mask_sin_superficie, 'surface_extracted'] = extracciones

# Estadísticas finales
extraidas = df_clean['surface_extracted'].notna().sum()
print(f"✅ Superficies extraídas: {extraidas:,}")
print(f"📈 Tasa de recuperación: {(extraidas/sin_superficie*100):.1f}%")
print(f"💎 Innovación text mining completada")

⛏️ TEXT MINING - SUPERFICIE OPTIMIZADA
----------------------------------------
🔍 Registros sin superficie: 273,094
✅ Superficies extraídas: 67,615
📈 Tasa de recuperación: 24.8%
💎 Innovación text mining completada


In [33]:
# ===============================================================
# ANÁLISIS DE EXTRACCIÓN ADICIONAL: ROOMS, BEDROOMS, BATHROOMS
# ===============================================================

print("🔍 ANÁLISIS DE EXTRACCIÓN ADICIONAL")
print("=" * 40)

# 1. Analizar qué información falta
print("📊 ESTADO ACTUAL DE VARIABLES:")
print("-" * 35)

variables_analizar = ['rooms', 'bedrooms', 'bathrooms']
for var in variables_analizar:
    if var in df_clean.columns:
        faltantes = df_clean[var].isna().sum()
        total = len(df_clean)
        pct = (faltantes/total)*100
        print(f"   {var:<12}: {faltantes:>8,} faltantes ({pct:>5.1f}%)")
    else:
        print(f"   {var:<12}: Columna no existe")

# 2. Muestras de descripciones para análisis
print(f"\n📝 EJEMPLOS DE DESCRIPCIONES (para análisis de patrones):")
print("-" * 60)

# Muestra aleatoria de descripciones
muestra_descripciones = df_clean['description'].dropna().sample(10, random_state=42)
for i, desc in enumerate(muestra_descripciones, 1):
    print(f"\n{i:2d}. {str(desc)[:200]}...")

# 3. Búsqueda de patrones específicos
print(f"\n\n🔎 BÚSQUEDA DE PATRONES EN DESCRIPCIONES:")
print("-" * 50)

# Patrones a buscar
patrones_busqueda = {
    'habitaciones': [
        r'(\d+)\s*(?:habitacion|dormitorio|cuarto|alcoba|recamara)',
        r'(?:habitacion|dormitorio|cuarto|alcoba|recamara)\s*(\d+)',
        r'(\d+)\s*hab',
        r'(\d+)\s*habs',
    ],
    'baños': [
        r'(\d+)\s*(?:baño|baños|bathroom|wc)',
        r'(?:baño|baños|bathroom|wc)\s*(\d+)',
        r'(\d+)\s*baths?',
    ],
    'habitaciones_totales': [
        r'(\d+)\s*(?:habitaciones?|cuartos?|rooms?)',
        r'(?:habitaciones?|cuartos?|rooms?)\s*(\d+)',
    ]
}

# Análisis en muestra más grande
muestra_analisis = df_clean['description'].dropna().sample(2000, random_state=42)

for categoria, patrones in patrones_busqueda.items():
    print(f"\n🔍 {categoria.upper()}:")
    for patron in patrones:
        matches = 0
        ejemplos = []
        for desc in muestra_analisis:
            if pd.notna(desc):
                encontrado = re.search(patron, str(desc).lower())
                if encontrado:
                    matches += 1
                    if len(ejemplos) < 3:  # Guardar pocos ejemplos
                        ejemplos.append((encontrado.group(1), str(desc)[:100]))
        
        print(f"   Patrón '{patron}': {matches} matches")
        for valor, ejemplo in ejemplos:
            print(f"      → {valor} en: {ejemplo}...")

🔍 ANÁLISIS DE EXTRACCIÓN ADICIONAL
📊 ESTADO ACTUAL DE VARIABLES:
-----------------------------------
   rooms       :  244,787 faltantes ( 88.1%)
   bedrooms    :  206,506 faltantes ( 74.3%)
   bathrooms   :   25,447 faltantes (  9.2%)

📝 EJEMPLOS DE DESCRIPCIONES (para análisis de patrones):
------------------------------------------------------------

 1. Codigo Inmueble 6620 Cómodo apartamento con 3 habitaciones, sala-comedor, cocina semi-integral, zona de ropas, 1 baño completo, iluminado, con buena ventilación. Su ambiente cálido con cómodos espacio...

 2. Excelente ubicación , buenas rutas de transporte, primer piso, 1 solo ambiente, 1 baño, 1 closet, cocineta,...

 3. Codigo Inmueble 561 Casa cerca al Éxito Laureles amplios espacios con muy buenas rutas de trabajo...

 4. Codigo Inmueble 5071 Apartamento con 2 alcobas, 2 closet, sala comedor, cocina integral mixta, 2 baños cabinados, zona de ropas,  calentador a gas, red de gas, balcón, piso madera y porcelanato, área ...

 5. 

In [34]:
# ===============================================================
# FUNCIONES DE EXTRACCIÓN PARA HABITACIONES Y BAÑOS
# ===============================================================

print("🔧 CREANDO FUNCIONES DE EXTRACCIÓN")
print("-" * 40)

def extraer_bedrooms(descripcion):
    """Extrae número de dormitorios/habitaciones desde descripción"""
    if pd.isna(descripcion):
        return None
    
    desc_lower = str(descripcion).lower()
    
    # Patrones ordenados por especificidad
    patrones = [
        r'(\d+)\s*(?:habitacion|dormitorio|alcoba|recamara)(?:es)?',
        r'(?:habitacion|dormitorio|alcoba|recamara)(?:es)?\s*(\d+)',
        r'(\d+)\s*hab(?:s)?[^a-z]',
        r'(\d+)\s*dorm(?:s)?[^a-z]',
        r'(\d+)\s*bed(?:room)?s?[^a-z]',
    ]
    
    for patron in patrones:
        matches = re.findall(patron, desc_lower)
        if matches:
            try:
                valor = int(matches[0])
                if 1 <= valor <= 10:  # Rango razonable
                    return valor
            except:
                continue
    return None

def extraer_bathrooms(descripcion):
    """Extrae número de baños desde descripción"""
    if pd.isna(descripcion):
        return None
    
    desc_lower = str(descripcion).lower()
    
    # Patrones para baños
    patrones = [
        r'(\d+)\s*(?:baño|baños|bathroom|wc)s?',
        r'(?:baño|baños|bathroom|wc)s?\s*(\d+)',
        r'(\d+)\s*bath(?:s)?[^a-z]',
        r'(\d+)\s*w\.?c\.?[^a-z]',
    ]
    
    for patron in patrones:
        matches = re.findall(patron, desc_lower)
        if matches:
            try:
                valor = int(matches[0])
                if 1 <= valor <= 8:  # Rango razonable
                    return valor
            except:
                continue
    return None

def extraer_rooms(descripcion):
    """Extrae número total de habitaciones/cuartos desde descripción"""
    if pd.isna(descripcion):
        return None
    
    desc_lower = str(descripcion).lower()
    
    # Patrones para habitaciones totales
    patrones = [
        r'(\d+)\s*(?:habitaciones?|cuartos?|rooms?)(?:\s|[^a-z])',
        r'(?:habitaciones?|cuartos?|rooms?)\s*(\d+)',
        r'(\d+)\s*(?:ambientes?)',
    ]
    
    for patron in patrones:
        matches = re.findall(patron, desc_lower)
        if matches:
            try:
                valor = int(matches[0])
                if 1 <= valor <= 15:  # Rango razonable
                    return valor
            except:
                continue
    return None

print("✅ Funciones de extracción creadas")

# Probar funciones en muestra pequeña
print(f"\n🧪 PRUEBA DE FUNCIONES:")
print("-" * 25)

muestra_test = df_clean['description'].dropna().sample(1000, random_state=42)

bedrooms_extraidos = muestra_test.apply(extraer_bedrooms).notna().sum()
bathrooms_extraidos = muestra_test.apply(extraer_bathrooms).notna().sum()
rooms_extraidos = muestra_test.apply(extraer_rooms).notna().sum()

print(f"Bedrooms extraídos: {bedrooms_extraidos}/1000 ({(bedrooms_extraidos/1000*100):.1f}%)")
print(f"Bathrooms extraídos: {bathrooms_extraidos}/1000 ({(bathrooms_extraidos/1000*100):.1f}%)")
print(f"Rooms extraídos: {rooms_extraidos}/1000 ({(rooms_extraidos/1000*100):.1f}%)")

🔧 CREANDO FUNCIONES DE EXTRACCIÓN
----------------------------------------
✅ Funciones de extracción creadas

🧪 PRUEBA DE FUNCIONES:
-------------------------
Bedrooms extraídos: 657/1000 (65.7%)
Bathrooms extraídos: 538/1000 (53.8%)
Rooms extraídos: 328/1000 (32.8%)


In [35]:
# ===============================================================
# APLICAR EXTRACCIÓN AL DATASET COMPLETO
# ===============================================================

print("🚀 APLICACIÓN AL DATASET COMPLETO")
print("=" * 40)

# Aplicar solo a registros que no tienen la información
variables_extraer = {
    'bedrooms': extraer_bedrooms,
    'bathrooms': extraer_bathrooms,
    'rooms': extraer_rooms
}

for variable, funcion in variables_extraer.items():
    print(f"\n🔄 Procesando {variable}...")
    
    # Verificar si la columna existe
    if variable in df_clean.columns:
        faltantes_inicial = df_clean[variable].isna().sum()
        mask_faltantes = df_clean[variable].isna()
    else:
        print(f"   Creando nueva columna '{variable}'")
        df_clean[variable] = np.nan
        faltantes_inicial = len(df_clean)
        mask_faltantes = df_clean[variable].isna()
    
    print(f"   Registros sin {variable}: {faltantes_inicial:,}")
    
    # Aplicar extracción
    extracciones = df_clean.loc[mask_faltantes, 'description'].apply(funcion)
    df_clean.loc[mask_faltantes, f'{variable}_extracted'] = extracciones
    
    # Estadísticas - CORREGIDO: calcular extraídos específicos para esta variable
    extraidos = (df_clean.loc[mask_faltantes, f'{variable}_extracted'].notna()).sum()
    tasa_recuperacion = (extraidos / faltantes_inicial * 100) if faltantes_inicial > 0 else 0
    
    print(f"   ✅ Extraídos: {extraidos:,}")
    print(f"   📈 Tasa recuperación: {tasa_recuperacion:.1f}%")

print(f"\n💎 EXTRACCIÓN ADICIONAL COMPLETADA")
print("=" * 40)

# Resumen final de extracciones
print(f"📊 RESUMEN DE TODAS LAS EXTRACCIONES:")
print("-" * 35)
print(f"   Surface: {df_clean['surface_extracted'].notna().sum():,} extraídas")
print(f"   Bedrooms: {df_clean['bedrooms_extracted'].notna().sum():,} extraídas")
print(f"   Bathrooms: {df_clean['bathrooms_extracted'].notna().sum():,} extraídas")
print(f"   Rooms: {df_clean['rooms_extracted'].notna().sum():,} extraídas")

total_extracciones = (
    df_clean['surface_extracted'].notna().sum() +
    df_clean['bedrooms_extracted'].notna().sum() +
    df_clean['bathrooms_extracted'].notna().sum() +
    df_clean['rooms_extracted'].notna().sum()
)

print(f"\n🎯 TOTAL DATOS RECUPERADOS: {total_extracciones:,}")
print(f"🏆 Text Mining: INNOVACIÓN COMPLETA")

🚀 APLICACIÓN AL DATASET COMPLETO

🔄 Procesando bedrooms...
   Registros sin bedrooms: 206,506
   ✅ Extraídos: 144,970
   📈 Tasa recuperación: 70.2%

🔄 Procesando bathrooms...
   Registros sin bathrooms: 25,447
   ✅ Extraídos: 12,824
   📈 Tasa recuperación: 50.4%

🔄 Procesando rooms...
   Registros sin rooms: 244,787
   ✅ Extraídos: 90,968
   📈 Tasa recuperación: 37.2%

💎 EXTRACCIÓN ADICIONAL COMPLETADA
📊 RESUMEN DE TODAS LAS EXTRACCIONES:
-----------------------------------
   Surface: 67,615 extraídas
   Bedrooms: 144,970 extraídas
   Bathrooms: 12,824 extraídas
   Rooms: 90,968 extraídas

🎯 TOTAL DATOS RECUPERADOS: 316,377
🏆 Text Mining: INNOVACIÓN COMPLETA


## 8️⃣ **Integración de Datos Extraídos**

**Decisión:** Consolidar datos originales con extracciones de text mining

**Justificación:** Crear variables finales que combinen datos originales con extracciones para maximizar completitud sin perder información original

In [36]:
# ===============================================================
# INTEGRACIÓN DE DATOS ORIGINALES CON EXTRACCIONES
# ===============================================================

print("🔗 INTEGRACIÓN DE DATOS")
print("=" * 30)

# Estrategia: Priorizar datos originales, completar con extracciones
variables_integrar = {
    'surface_total': 'surface_extracted',
    'bedrooms': 'bedrooms_extracted', 
    'bathrooms': 'bathrooms_extracted',
    'rooms': 'rooms_extracted'
}

for original, extraida in variables_integrar.items():
    if extraida in df_clean.columns:
        # Estado antes de integración
        antes_faltantes = df_clean[original].isna().sum()
        
        # Integrar: usar original si existe, sino usar extraída
        df_clean[f'{original}_final'] = df_clean[original].fillna(df_clean[extraida])
        
        # Estado después de integración
        despues_faltantes = df_clean[f'{original}_final'].isna().sum()
        completados = antes_faltantes - despues_faltantes
        
        print(f"\n📊 {original.upper()}:")
        print(f"   Antes: {antes_faltantes:,} faltantes")
        print(f"   Después: {despues_faltantes:,} faltantes")
        print(f"   ✅ Completados: {completados:,}")
        if antes_faltantes > 0:
            print(f"   📈 Mejora: {(completados/antes_faltantes*100):.1f}%")

print(f"\n🎯 RESUMEN DE INTEGRACIÓN:")
print("-" * 30)

# Calcular total de completaciones
total_completaciones = 0
for original, extraida in variables_integrar.items():
    if extraida in df_clean.columns:
        antes = df_clean[original].isna().sum()
        despues = df_clean[f'{original}_final'].isna().sum()
        total_completaciones += (antes - despues)

print(f"📈 Total valores completados: {total_completaciones:,}")
print(f"💎 Integración exitosa completada")

🔗 INTEGRACIÓN DE DATOS

📊 SURFACE_TOTAL:
   Antes: 273,094 faltantes
   Después: 205,479 faltantes
   ✅ Completados: 67,615
   📈 Mejora: 24.8%

📊 BEDROOMS:
   Antes: 206,506 faltantes
   Después: 61,536 faltantes
   ✅ Completados: 144,970
   📈 Mejora: 70.2%

📊 BATHROOMS:
   Antes: 25,447 faltantes
   Después: 12,623 faltantes
   ✅ Completados: 12,824
   📈 Mejora: 50.4%

📊 ROOMS:
   Antes: 244,787 faltantes
   Después: 153,819 faltantes
   ✅ Completados: 90,968
   📈 Mejora: 37.2%

🎯 RESUMEN DE INTEGRACIÓN:
------------------------------
📈 Total valores completados: 316,377
💎 Integración exitosa completada


## 9️⃣ **Tratamiento de Valores Faltantes**

**Objetivo:** Imputar valores faltantes con estrategias estadísticas antes de la imputación avanzada de superficie

**Justificación:** Completar variables críticas como coordenadas y ubicación para maximizar la calidad del dataset antes del modelado ML de superficie

In [38]:
# ===============================================================
# ANÁLISIS DE VALORES FALTANTES RESTANTES
# ===============================================================

print("🔍 ANÁLISIS DE VALORES FALTANTES RESTANTES")
print("=" * 50)

# Evaluar estado actual después de text mining e integración
variables_criticas = ['lat', 'lon', 'l4', 'surface_total_final', 'bedrooms_final', 'bathrooms_final']

print("📊 ESTADO ACTUAL DE VARIABLES CRÍTICAS:")
print("-" * 45)

for var in variables_criticas:
    if var in df_clean.columns:
        faltantes = df_clean[var].isna().sum()
        total = len(df_clean)
        pct = (faltantes / total) * 100
        print(f"   {var:<20}: {faltantes:>8,} faltantes ({pct:>5.1f}%)")

# Identificar variables para imputación simple
print(f"\n🎯 ESTRATEGIAS DE IMPUTACIÓN:")
print("-" * 35)

# 1. Coordenadas faltantes - imputar por centroide de barrio
coords_faltantes = df_clean[['lat', 'lon']].isna().any(axis=1).sum()
barrios_disponibles = df_clean['l4'].notna().sum()
print(f"   Coordenadas: {coords_faltantes:,} faltantes")
print(f"   Barrios disponibles: {barrios_disponibles:,}")

# 2. Barrios faltantes - estrategia compleja (mantener para análisis)
barrios_faltantes = df_clean['l4'].isna().sum()
print(f"   Barrios: {barrios_faltantes:,} faltantes")

print(f"\n✅ ANÁLISIS COMPLETADO - LISTO PARA IMPUTACIÓN")

🔍 ANÁLISIS DE VALORES FALTANTES RESTANTES
📊 ESTADO ACTUAL DE VARIABLES CRÍTICAS:
---------------------------------------------
   lat                 :  163,274 faltantes ( 58.8%)
   lon                 :  163,274 faltantes ( 58.8%)
   l4                  :  211,876 faltantes ( 76.3%)
   surface_total_final :  205,479 faltantes ( 74.0%)
   bedrooms_final      :   61,536 faltantes ( 22.1%)
   bathrooms_final     :   12,623 faltantes (  4.5%)

🎯 ESTRATEGIAS DE IMPUTACIÓN:
-----------------------------------
   Coordenadas: 163,274 faltantes
   Barrios disponibles: 65,980
   Barrios: 211,876 faltantes

✅ ANÁLISIS COMPLETADO - LISTO PARA IMPUTACIÓN


## 🔟 **Extracción de Ubicación por Text Mining** 🌍

**Objetivo:** Extraer información de ubicación (barrios, sectores, zonas) desde las descripciones

**Justificación:** Con 211,876 registros sin barrio (l4) pero con descripciones disponibles, esta estrategia puede recuperar información geográfica valiosa que está "escondida" en el texto libre.

**Estrategia:** Usar patrones específicos para identificar menciones de barrios, sectores y zonas conocidas de Antioquia

In [39]:
# ===============================================================
# ANÁLISIS DE DESCRIPCIONES PARA EXTRACCIÓN DE UBICACIÓN
# ===============================================================

print("🌍 EXTRACCIÓN DE UBICACIÓN POR TEXT MINING")
print("=" * 50)

# 1. Estado actual de ubicaciones
print("📊 ESTADO ACTUAL DE UBICACIÓN:")
print("-" * 35)

total_registros = len(df_clean)
con_barrio = df_clean['l4'].notna().sum()
sin_barrio = df_clean['l4'].isna().sum()
con_descripcion = df_clean['description'].notna().sum()

print(f"Total registros: {total_registros:,}")
print(f"Con barrio (l4): {con_barrio:,} ({(con_barrio/total_registros*100):.1f}%)")
print(f"Sin barrio (l4): {sin_barrio:,} ({(sin_barrio/total_registros*100):.1f}%)")
print(f"Con descripción: {con_descripcion:,} ({(con_descripcion/total_registros*100):.1f}%)")

# 2. Candidatos para extracción: sin barrio PERO con descripción
candidatos_ubicacion = (df_clean['l4'].isna() & df_clean['description'].notna()).sum()
print(f"\n🎯 Candidatos para extracción: {candidatos_ubicacion:,}")
print(f"   (Sin barrio PERO con descripción)")

# 3. Obtener barrios conocidos para crear patrones
print(f"\n📋 BARRIOS CONOCIDOS EN EL DATASET:")
print("-" * 35)

barrios_conocidos = df_clean['l4'].dropna().unique()
print(f"Total barrios únicos: {len(barrios_conocidos):,}")

# Mostrar los más comunes
top_barrios = df_clean['l4'].value_counts().head(20)
print(f"\nTop 20 barrios más comunes:")
for i, (barrio, count) in enumerate(top_barrios.items(), 1):
    print(f"  {i:2d}. {barrio:<30}: {count:>4,} propiedades")

🌍 EXTRACCIÓN DE UBICACIÓN POR TEXT MINING
📊 ESTADO ACTUAL DE UBICACIÓN:
-----------------------------------
Total registros: 277,856
Con barrio (l4): 65,980 (23.7%)
Sin barrio (l4): 211,876 (76.3%)
Con descripción: 277,597 (99.9%)

🎯 Candidatos para extracción: 211,706
   (Sin barrio PERO con descripción)

📋 BARRIOS CONOCIDOS EN EL DATASET:
-----------------------------------
Total barrios únicos: 21

Top 20 barrios más comunes:
   1. El Poblado                    : 21,896 propiedades
   2. Laureles                      : 11,039 propiedades
   3. Belén                         : 8,897 propiedades
   4. La América                    : 5,747 propiedades
   5. Robledo                       : 3,487 propiedades
   6. Buenos Aires                  : 3,306 propiedades
   7. Candelaria                    : 3,291 propiedades
   8. San Javier                    : 1,177 propiedades
   9. Guayabal                      : 1,036 propiedades
  10. Castilla                      : 1,010 propiedades
  11.

In [40]:
# ===============================================================
# ANÁLISIS DE PATRONES EN DESCRIPCIONES
# ===============================================================

print("🔍 ANÁLISIS DE PATRONES DE UBICACIÓN EN DESCRIPCIONES")
print("=" * 60)

# Muestra de descripciones de registros SIN barrio para análisis
sin_barrio_mask = df_clean['l4'].isna() & df_clean['description'].notna()
muestra_sin_barrio = df_clean[sin_barrio_mask]['description'].sample(15, random_state=42)

print("📝 MUESTRA DE DESCRIPCIONES SIN BARRIO:")
print("-" * 45)
for i, desc in enumerate(muestra_sin_barrio, 1):
    desc_corta = str(desc)[:120] + "..." if len(str(desc)) > 120 else str(desc)
    print(f"\n{i:2d}. {desc_corta}")

# Búsqueda específica de barrios conocidos en descripciones
print(f"\n\n🎯 BÚSQUEDA DE BARRIOS CONOCIDOS EN DESCRIPCIONES:")
print("-" * 55)

# ANÁLISIS DE CIUDADES DISPONIBLES EN EL DATASET
print(f"\n🏙️ ANÁLISIS DE CIUDADES DISPONIBLES:")
print("-" * 40)

# Obtener ciudades (l3) del dataset actual
ciudades_disponibles = df_clean['l3'].dropna().unique()
conteo_ciudades = df_clean['l3'].value_counts()

print(f"Total ciudades únicas: {len(ciudades_disponibles)}")
print(f"\nCiudades ordenadas por cantidad de propiedades:")
for i, (ciudad, count) in enumerate(conteo_ciudades.items(), 1):
    pct = (count / len(df_clean)) * 100
    print(f"  {i:2d}. {ciudad:<25}: {count:>6,} ({pct:>5.1f}%)")

# Crear lista de ciudades para buscar (basada en el dataset real)
ciudades_buscar = list(ciudades_disponibles)
print(f"\n🎯 Lista de ciudades para extracción: {len(ciudades_buscar)} ciudades")

# Buscar cada ciudad en descripciones SIN ciudad asignada
sin_ciudad_mask = df_clean['l3'].isna() & df_clean['description'].notna()
registros_sin_ciudad = sin_ciudad_mask.sum()
print(f"📊 Registros sin ciudad (l3) pero con descripción: {registros_sin_ciudad:,}")

# Analizar una muestra más pequeña para ciudades
muestra_sin_ciudad = df_clean[sin_ciudad_mask]['description'].sample(min(50, registros_sin_ciudad), random_state=42)

encontrados_ciudades = {}
for ciudad in ciudades_buscar:
    # Patrón flexible para encontrar la ciudad
    patron = rf'\b{re.escape(ciudad.lower())}\b'
    
    # Buscar en descripciones (convertir a minúsculas)
    matches = 0
    ejemplos = []
    
    for desc in muestra_sin_ciudad:
        if pd.notna(desc):
            desc_lower = str(desc).lower()
            if re.search(patron, desc_lower):
                matches += 1
                if len(ejemplos) < 2:  # Máximo 2 ejemplos
                    ejemplos.append(str(desc)[:100] + "...")
    
    if matches > 0:
        encontrados_ciudades[ciudad] = {'matches': matches, 'ejemplos': ejemplos}

# Mostrar resultados de ciudades
print(f"\nCiudades encontradas en muestra de {len(muestra_sin_ciudad)} descripciones:")
for ciudad, info in encontrados_ciudades.items():
    print(f"\n• {ciudad}: {info['matches']} matches")
    for ejemplo in info['ejemplos']:
        print(f"  → {ejemplo}")

print(f"\n✅ Análisis de ciudades completado")
print(f"📊 Ciudades detectadas: {len(encontrados_ciudades)} de {len(ciudades_buscar)} buscadas")

🔍 ANÁLISIS DE PATRONES DE UBICACIÓN EN DESCRIPCIONES
📝 MUESTRA DE DESCRIPCIONES SIN BARRIO:
---------------------------------------------

 1. Codigo Inmueble 505512 Excelente casa de dos niveles, la cual cuenta con 6 habitaciones. 5 baños, parqueadero doble, amp...

 2. Apto 501: 
80 mt2
Estrato 3
Dos años de antiguedad
Predial (semestral): $26.800
No paga admon 
- Alcoba principal con ba...

 3. Apartamento para arriendo en Campus Reservado 
Cuenta con: 2 habitaciones, 2 baños, 1 closet, 1 Vestier, calentador, sal...

 4. Codigo Inmueble 504708 Acogedora casa ubicado en un sector muy residencial con cómodos espacios buena distribución y exc...

 5. Codigo Inmueble 340 Apartamento para arriendo o Venta, tercer piso, parqueadero cubierto, citofonía, cerca a unicentro, ...

 6. Hermoso apartamento de 114 M2, tres habitaciones, tres baños, sala comedor, cocina integral abierta, zona de ropas, vist...

 7. Codigo Inmueble 718 Arrienda apartamento 75 m2 , 3 alcobas , 2 closet, 1 baño cabin

In [41]:
# ===============================================================
# FUNCIONES DE EXTRACCIÓN: CIUDAD Y BARRIO
# ===============================================================

print("🏗️ CREANDO FUNCIONES DE EXTRACCIÓN DE UBICACIÓN")
print("=" * 55)

def extraer_ciudad(descripcion):
    """Extrae ciudad desde descripción usando ciudades conocidas del dataset"""
    if pd.isna(descripcion):
        return None
    
    desc_lower = str(descripcion).lower()
    
    # Lista de ciudades del dataset (en orden de frecuencia)
    ciudades_buscar = [
        'medellín', 'medellin', 'envigado', 'itagüí', 'itagui', 'sabaneta', 
        'bello', 'copacabana', 'la estrella', 'estrella', 'caldas', 
        'girardota', 'barbosa', 'rionegro'
    ]
    
    # Buscar cada ciudad con patrones específicos
    for ciudad in ciudades_buscar:
        # Patrón que busca la ciudad como palabra completa
        patron = rf'\b{re.escape(ciudad)}\b'
        
        if re.search(patron, desc_lower):
            # Mapear a nombre estándar del dataset
            if ciudad in ['medellin', 'medellín']:
                return 'Medellín'
            elif ciudad in ['itagui', 'itagüí']:
                return 'Itagüí'
            elif ciudad in ['estrella', 'la estrella']:
                return 'La Estrella'
            else:
                return ciudad.title()
    
    return None

def extraer_barrio(descripcion):
    """Extrae barrio desde descripción usando barrios conocidos del dataset"""
    if pd.isna(descripcion):
        return None
    
    desc_lower = str(descripcion).lower()
    
    # Lista de barrios conocidos del dataset (solo barrios reales, no ciudades)
    barrios_buscar = [
        'el poblado', 'poblado', 'laureles', 'belén', 'belen', 'la américa', 'america',
        'robledo', 'buenos aires', 'candelaria', 'san javier', 'guayabal', 'castilla',
        'san cristóbal', 'san cristobal', 'aranjuez', 'altavista', 'villa hermosa',
        'santa elena', 'san antonio de prado', 'manrique', 'doce de octubre', 
        'santa cruz', 'popular'
    ]
    
    # Buscar cada barrio con patrones específicos
    for barrio in barrios_buscar:
        # Patrón que busca el barrio como palabra completa
        patron = rf'\b{re.escape(barrio)}\b'
        
        if re.search(patron, desc_lower):
            # Mapear a nombre estándar del dataset
            if barrio in ['poblado', 'el poblado']:
                return 'El Poblado'
            elif barrio in ['belen', 'belén']:
                return 'Belén'
            elif barrio in ['america', 'la américa']:
                return 'La América'
            elif barrio in ['san cristobal', 'san cristóbal']:
                return 'San Cristóbal'
            else:
                return barrio.title()
    
    return None

print("✅ Funciones de extracción creadas")

# Probar funciones en muestra pequeña
print(f"\n🧪 PRUEBA DE FUNCIONES DE UBICACIÓN:")
print("-" * 40)

# Muestra para probar
muestra_test_ubicacion = df_clean['description'].dropna().sample(1000, random_state=42)

ciudades_extraidas = muestra_test_ubicacion.apply(extraer_ciudad).notna().sum()
barrios_extraidos = muestra_test_ubicacion.apply(extraer_barrio).notna().sum()

print(f"Ciudades extraídas: {ciudades_extraidas}/1000 ({(ciudades_extraidas/1000*100):.1f}%)")
print(f"Barrios extraídos: {barrios_extraidos}/1000 ({(barrios_extraidos/1000*100):.1f}%)")

# Mostrar ejemplos de extracciones exitosas
print(f"\n📋 EJEMPLOS DE EXTRACCIONES EXITOSAS:")
print("-" * 40)

ejemplos_ciudad = muestra_test_ubicacion[muestra_test_ubicacion.apply(extraer_ciudad).notna()].head(3)
ejemplos_barrio = muestra_test_ubicacion[muestra_test_ubicacion.apply(extraer_barrio).notna()].head(3)

print("🏙️ CIUDADES:")
for i, desc in enumerate(ejemplos_ciudad, 1):
    ciudad_encontrada = extraer_ciudad(desc)
    print(f"  {i}. Ciudad: {ciudad_encontrada}")
    print(f"     Descripción: {str(desc)[:80]}...")

print("\n🏘️ BARRIOS:")
for i, desc in enumerate(ejemplos_barrio, 1):
    barrio_encontrado = extraer_barrio(desc)
    print(f"  {i}. Barrio: {barrio_encontrado}")
    print(f"     Descripción: {str(desc)[:80]}...")

print(f"\n✅ PRUEBAS COMPLETADAS")

🏗️ CREANDO FUNCIONES DE EXTRACCIÓN DE UBICACIÓN
✅ Funciones de extracción creadas

🧪 PRUEBA DE FUNCIONES DE UBICACIÓN:
----------------------------------------
Ciudades extraídas: 233/1000 (23.3%)
Barrios extraídos: 183/1000 (18.3%)

📋 EJEMPLOS DE EXTRACCIONES EXITOSAS:
----------------------------------------
🏙️ CIUDADES:
  1. Ciudad: Medellín
     Descripción: ¿Estás buscando pent-house, con excelente ubicación iluminacion natural y un amp...
  2. Ciudad: Itagüí
     Descripción: 622-14137 Apartamento en arriendo ubicado en Itagüí sector Suramérica.Excelentes...
  3. Ciudad: La Estrella
     Descripción: <b>Suramerica, La Estrella, arriendo apartamento</b><br><br>Arriendo apartamento...

🏘️ BARRIOS:
  1. Barrio: Laureles
     Descripción: Codigo Inmueble 561 Casa cerca al Éxito Laureles amplios espacios con muy buenas...
  2. Barrio: Laureles
     Descripción: <b>202 LAURELES 450</b><br><br>Se vende c&oacute;modo y lindo apartamento de 110...
  3. Barrio: El Poblado
     Descripción:

In [42]:
# ===============================================================
# APLICAR EXTRACCIÓN DE UBICACIÓN AL DATASET COMPLETO
# ===============================================================

print("🚀 APLICACIÓN AL DATASET COMPLETO - UBICACIÓN")
print("=" * 50)

# PASO 1: EXTRACCIÓN DE CIUDADES (l3)
print("\n🏙️ EXTRACCIÓN DE CIUDADES:")
print("-" * 35)

# Estado inicial de ciudades
ciudades_faltantes_inicial = df_clean['l3'].isna().sum()
print(f"Ciudades faltantes antes: {ciudades_faltantes_inicial:,}")

# Aplicar extracción solo a registros sin ciudad pero con descripción
mask_sin_ciudad = df_clean['l3'].isna() & df_clean['description'].notna()
candidatos_ciudad = mask_sin_ciudad.sum()
print(f"Candidatos para extracción: {candidatos_ciudad:,}")

# Extraer ciudades
extracciones_ciudad = df_clean.loc[mask_sin_ciudad, 'description'].apply(extraer_ciudad)
df_clean.loc[mask_sin_ciudad, 'l3_extracted'] = extracciones_ciudad

# Estadísticas de ciudades
ciudades_extraidas = df_clean['l3_extracted'].notna().sum()
tasa_ciudad = (ciudades_extraidas / candidatos_ciudad * 100) if candidatos_ciudad > 0 else 0

print(f"✅ Ciudades extraídas: {ciudades_extraidas:,}")
print(f"📈 Tasa de éxito: {tasa_ciudad:.1f}%")

# PASO 2: EXTRACCIÓN DE BARRIOS (l4)
print(f"\n🏘️ EXTRACCIÓN DE BARRIOS:")
print("-" * 35)

# Estado inicial de barrios
barrios_faltantes_inicial = df_clean['l4'].isna().sum()
print(f"Barrios faltantes antes: {barrios_faltantes_inicial:,}")

# Aplicar extracción solo a registros sin barrio pero con descripción
mask_sin_barrio = df_clean['l4'].isna() & df_clean['description'].notna()
candidatos_barrio = mask_sin_barrio.sum()
print(f"Candidatos para extracción: {candidatos_barrio:,}")

# Extraer barrios
extracciones_barrio = df_clean.loc[mask_sin_barrio, 'description'].apply(extraer_barrio)
df_clean.loc[mask_sin_barrio, 'l4_extracted'] = extracciones_barrio

# Estadísticas de barrios
barrios_extraidos = df_clean['l4_extracted'].notna().sum()
tasa_barrio = (barrios_extraidos / candidatos_barrio * 100) if candidatos_barrio > 0 else 0

print(f"✅ Barrios extraídos: {barrios_extraidos:,}")
print(f"📈 Tasa de éxito: {tasa_barrio:.1f}%")

# PASO 3: INTEGRACIÓN DE UBICACIONES
print(f"\n🔗 INTEGRACIÓN DE UBICACIONES:")
print("-" * 35)

# Integrar ciudades: usar original si existe, sino usar extraída
df_clean['l3_final'] = df_clean['l3'].fillna(df_clean['l3_extracted'])

# Integrar barrios: usar original si existe, sino usar extraída  
df_clean['l4_final'] = df_clean['l4'].fillna(df_clean['l4_extracted'])

# Estadísticas finales
ciudades_final = df_clean['l3_final'].notna().sum()
barrios_final = df_clean['l4_final'].notna().sum()

ciudades_completadas = ciudades_final - (len(df_clean) - ciudades_faltantes_inicial)
barrios_completados = barrios_final - (len(df_clean) - barrios_faltantes_inicial)

print(f"📊 RESULTADOS FINALES:")
print("-" * 25)
print(f"Ciudades completadas: {ciudades_completadas:,}")
print(f"Barrios completados: {barrios_completados:,}")

# Mostrar distribución de ciudades y barrios extraídos
print(f"\n📋 DISTRIBUCIÓN DE EXTRACCIONES:")
print("-" * 35)

if 'l3_extracted' in df_clean.columns:
    print("🏙️ Ciudades extraídas:")
    ciudades_extraidas_dist = df_clean['l3_extracted'].value_counts().head(10)
    for ciudad, count in ciudades_extraidas_dist.items():
        print(f"   {ciudad:<20}: {count:>4,}")

if 'l4_extracted' in df_clean.columns:
    print("\n🏘️ Barrios extraídos:")
    barrios_extraidos_dist = df_clean['l4_extracted'].value_counts().head(10)
    for barrio, count in barrios_extraidos_dist.items():
        print(f"   {barrio:<20}: {count:>4,}")

print(f"\n🎯 TOTAL UBICACIONES RECUPERADAS: {ciudades_extraidas + barrios_extraidos:,}")
print(f"🏆 Extracción de ubicación: COMPLETADA")

🚀 APLICACIÓN AL DATASET COMPLETO - UBICACIÓN

🏙️ EXTRACCIÓN DE CIUDADES:
-----------------------------------
Ciudades faltantes antes: 3,689
Candidatos para extracción: 3,649
✅ Ciudades extraídas: 1,019
📈 Tasa de éxito: 27.9%

🏘️ EXTRACCIÓN DE BARRIOS:
-----------------------------------
Barrios faltantes antes: 211,876
Candidatos para extracción: 211,706
✅ Barrios extraídos: 25,775
📈 Tasa de éxito: 12.2%

🔗 INTEGRACIÓN DE UBICACIONES:
-----------------------------------
📊 RESULTADOS FINALES:
-------------------------
Ciudades completadas: 1,019
Barrios completados: 25,775

📋 DISTRIBUCIÓN DE EXTRACCIONES:
-----------------------------------
🏙️ Ciudades extraídas:
   Rionegro            :  516
   Medellín            :  445
   Envigado            :   25
   Sabaneta            :   15
   La Estrella         :    8
   Bello               :    6
   Barbosa             :    2
   Copacabana          :    1
   Caldas              :    1

🏘️ Barrios extraídos:
   El Poblado          : 9,107
   B

In [43]:
# ===============================================================
# VALIDACIÓN Y RESUMEN DE EXTRACCIÓN DE UBICACIÓN
# ===============================================================

print("🔍 VALIDACIÓN DE EXTRACCIÓN DE UBICACIÓN")
print("=" * 50)

# 1. Comparar estado antes vs después
print("📊 COMPARACIÓN ANTES vs DESPUÉS:")
print("-" * 35)

# Ciudades
ciudad_antes = df_clean['l3'].notna().sum()
ciudad_despues = df_clean['l3_final'].notna().sum()
mejora_ciudad = ciudad_despues - ciudad_antes

print(f"🏙️ CIUDADES:")
print(f"   Antes: {ciudad_antes:,}")
print(f"   Después: {ciudad_despues:,}")
print(f"   Mejora: +{mejora_ciudad:,} ({(mejora_ciudad/len(df_clean)*100):.1f}%)")

# Barrios  
barrio_antes = df_clean['l4'].notna().sum()
barrio_despues = df_clean['l4_final'].notna().sum()
mejora_barrio = barrio_despues - barrio_antes

print(f"\n🏘️ BARRIOS:")
print(f"   Antes: {barrio_antes:,}")
print(f"   Después: {barrio_despues:,}")
print(f"   Mejora: +{mejora_barrio:,} ({(mejora_barrio/len(df_clean)*100):.1f}%)")

# 2. Impacto en coordenadas faltantes
print(f"\n🗺️ IMPACTO EN COORDENADAS:")
print("-" * 30)

# Recalcular potencial de imputación de coordenadas
coords_faltantes = df_clean[['lat', 'lon']].isna().any(axis=1).sum()
barrios_disponibles_ahora = df_clean['l4_final'].notna().sum()

# Casos donde ahora PODRÍAMOS imputar coordenadas
sin_coords_con_barrio_ahora = (
    df_clean[['lat', 'lon']].isna().any(axis=1) & 
    df_clean['l4_final'].notna()
).sum()

print(f"Coordenadas faltantes: {coords_faltantes:,}")
print(f"Barrios disponibles ahora: {barrios_disponibles_ahora:,}")
print(f"🎯 Casos para imputar coords: {sin_coords_con_barrio_ahora:,}")

# 3. Muestra de casos exitosos
print(f"\n📋 MUESTRA DE CASOS EXITOSOS:")
print("-" * 35)

# Casos donde se recuperó barrio
casos_barrio_recuperado = df_clean[
    df_clean['l4'].isna() & 
    df_clean['l4_extracted'].notna()
][['description', 'l4_extracted']].head(5)

print("🏘️ Barrios recuperados:")
for idx, row in casos_barrio_recuperado.iterrows():
    desc = str(row['description'])[:80] + "..."
    barrio = row['l4_extracted']
    print(f"   → {barrio}: {desc}")

# Casos donde se recuperó ciudad
casos_ciudad_recuperada = df_clean[
    df_clean['l3'].isna() & 
    df_clean['l3_extracted'].notna()
][['description', 'l3_extracted']].head(3)

print(f"\n🏙️ Ciudades recuperadas:")
for idx, row in casos_ciudad_recuperada.iterrows():
    desc = str(row['description'])[:80] + "..."
    ciudad = row['l3_extracted']
    print(f"   → {ciudad}: {desc}")

print(f"\n✅ VALIDACIÓN COMPLETADA")
print(f"🏆 EXTRACCIÓN DE UBICACIÓN: ÉXITO TOTAL")
print(f"💎 Innovación de Text Mining: {mejora_ciudad + mejora_barrio:,} ubicaciones recuperadas")

🔍 VALIDACIÓN DE EXTRACCIÓN DE UBICACIÓN
📊 COMPARACIÓN ANTES vs DESPUÉS:
-----------------------------------
🏙️ CIUDADES:
   Antes: 274,167
   Después: 275,186
   Mejora: +1,019 (0.4%)

🏘️ BARRIOS:
   Antes: 65,980
   Después: 91,755
   Mejora: +25,775 (9.3%)

🗺️ IMPACTO EN COORDENADAS:
------------------------------
Coordenadas faltantes: 163,274
Barrios disponibles ahora: 91,755
🎯 Casos para imputar coords: 23,496

📋 MUESTRA DE CASOS EXITOSOS:
-----------------------------------
🏘️ Barrios recuperados:
   → El Poblado: Codigo Inmueble 5980 Hermosa casa para alquilar en zona residencial de Medellin,...
   → Belén: Codigo Inmueble 502031 CASA EN VENTA EN EL SECTOR DE BELÉN FATIMA CON 4 ALCOBAS,...
   → Laureles: Codigo Inmueble 561 Casa cerca al Éxito Laureles amplios espacios con muy buenas...
   → Laureles: Codigo Inmueble 608 Se vende casa en Laureles Almeria 120 mts. Sala comedor + ba...
   → Laureles: Codigo Inmueble 561 Casa cerca al Éxito Laureles amplios espacios con muy buenas.

In [44]:
# ===============================================================
# ANÁLISIS DEL CAMPO TITLE PARA EXTRACCIÓN DE BARRIOS
# ===============================================================

print("📰 ANÁLISIS DEL CAMPO TITLE PARA BARRIOS")
print("=" * 50)

# 1. Estado del campo title
print("📊 ESTADO DEL CAMPO TITLE:")
print("-" * 30)

title_disponible = df_clean['title'].notna().sum()
title_faltante = df_clean['title'].isna().sum()
total = len(df_clean)

print(f"Total registros: {total:,}")
print(f"Con title: {title_disponible:,} ({(title_disponible/total*100):.1f}%)")
print(f"Sin title: {title_faltante:,} ({(title_faltante/total*100):.1f}%)")

# 2. Candidatos para extracción desde title
candidatos_title = (
    df_clean['l4_final'].isna() & 
    df_clean['title'].notna()
).sum()

print(f"\n🎯 Candidatos title (sin barrio pero con title): {candidatos_title:,}")

# 3. Muestra de títulos para análisis
print(f"\n📝 MUESTRA DE TÍTULOS SIN BARRIO:")
print("-" * 35)

sin_barrio_con_title = df_clean[
    df_clean['l4_final'].isna() & 
    df_clean['title'].notna()
]['title'].sample(15, random_state=42)

for i, title in enumerate(sin_barrio_con_title, 1):
    title_corto = str(title)[:80] + "..." if len(str(title)) > 80 else str(title)
    print(f"\n{i:2d}. {title_corto}")

# 4. Búsqueda de barrios en títulos
print(f"\n\n🔍 BÚSQUEDA DE BARRIOS EN TÍTULOS:")
print("-" * 40)

# Lista de barrios conocidos (misma que usamos antes)
barrios_buscar_title = [
    'el poblado', 'poblado', 'laureles', 'belén', 'belen', 'la américa', 'america',
    'robledo', 'buenos aires', 'candelaria', 'san javier', 'guayabal', 'castilla',
    'san cristóbal', 'san cristobal', 'aranjuez', 'altavista', 'villa hermosa',
    'santa elena', 'san antonio de prado', 'manrique', 'doce de octubre', 
    'santa cruz', 'popular'
]

encontrados_title = {}
for barrio in barrios_buscar_title:
    # Patrón flexible para encontrar el barrio en título
    patron = rf'\b{re.escape(barrio)}\b'
    
    # Buscar en títulos SIN barrio
    matches = 0
    ejemplos = []
    
    for title in sin_barrio_con_title:
        if pd.notna(title):
            title_lower = str(title).lower()
            if re.search(patron, title_lower):
                matches += 1
                if len(ejemplos) < 2:  # Máximo 2 ejemplos
                    ejemplos.append(str(title)[:60] + "...")
    
    if matches > 0:
        encontrados_title[barrio] = {'matches': matches, 'ejemplos': ejemplos}

# Mostrar resultados
print(f"Barrios encontrados en muestra de {len(sin_barrio_con_title)} títulos:")
for barrio, info in encontrados_title.items():
    print(f"\n• {barrio}: {info['matches']} matches")
    for ejemplo in info['ejemplos']:
        print(f"  → {ejemplo}")

print(f"\n✅ Análisis de títulos completado")
print(f"📊 Barrios detectados en titles: {len(encontrados_title)} de {len(barrios_buscar_title)} buscados")

📰 ANÁLISIS DEL CAMPO TITLE PARA BARRIOS
📊 ESTADO DEL CAMPO TITLE:
------------------------------
Total registros: 277,856
Con title: 277,855 (100.0%)
Sin title: 1 (0.0%)

🎯 Candidatos title (sin barrio pero con title): 186,100

📝 MUESTRA DE TÍTULOS SIN BARRIO:
-----------------------------------

 1. VENDO APARTAMENTO AVES MARIAS SABANETA COD. 900871

 2. Apartamento en Venta Ubicado en MEDELLIN

 3. Apartamento en Arriendo Ubicado en SABANETA

 4. Apartamento en Arriendo Ubicado en MEDELLIN

 5. PR 12140 SE ARRIENDA APARTAMENTO EN SECTOR JARDINES _ ENVIGADO

 6. APARTAMENTO VENTA EL RETIRO ANTIOQUIA COD: 15101 _ wasi1260235

 7. Casa en Venta Ubicado en MEDELLIN

 8. Apartamento en Arriendo Ubicado en MEDELLIN

 9. Apartamento en Arriendo Ubicado en SABANETA

10. Loma del Escobero, venta apartamento

11. Casa en venta 90m2 Niquia Bello

12. Apartamento en Venta Ubicado en MEDELLIN

13. Apartamento en Arriendo Ubicado en RIONEGRO

14. Apartamento en arriendo en Rionegro (Antioquia) La 

In [45]:
# ===============================================================
# FUNCIÓN DE EXTRACCIÓN DESDE TITLES (CIUDADES Y SECTORES)
# ===============================================================

print("🔧 CREANDO FUNCIÓN DE EXTRACCIÓN DESDE TITLES")
print("=" * 50)

def extraer_ciudad_desde_title(title):
    """Extrae ciudad desde title usando patrones específicos"""
    if pd.isna(title):
        return None
    
    title_lower = str(title).lower()
    
    # Ciudades del área metropolitana
    ciudades_title = [
        'medellín', 'medellin', 'envigado', 'itagüí', 'itagui', 'sabaneta', 
        'bello', 'copacabana', 'la estrella', 'estrella', 'caldas', 
        'girardota', 'barbosa', 'rionegro'
    ]
    
    for ciudad in ciudades_title:
        patron = rf'\b{re.escape(ciudad)}\b'
        if re.search(patron, title_lower):
            # Mapear a nombre estándar
            if ciudad in ['medellin', 'medellín']:
                return 'Medellín'
            elif ciudad in ['itagui', 'itagüí']:
                return 'Itagüí'
            elif ciudad in ['estrella', 'la estrella']:
                return 'La Estrella'
            else:
                return ciudad.title()
    
    return None

def extraer_barrio_desde_title(title):
    """Extrae barrio/sector desde title con patrones específicos"""
    if pd.isna(title):
        return None
    
    title_lower = str(title).lower()
    
    # Barrios y sectores comunes en titles
    ubicaciones_title = [
        # Barrios conocidos
        'el poblado', 'poblado', 'laureles', 'belén', 'belen', 'la américa', 'america',
        'robledo', 'buenos aires', 'candelaria', 'san javier', 'guayabal', 'castilla',
        'san cristóbal', 'san cristobal', 'aranjuez', 'altavista', 'villa hermosa',
        'santa elena', 'san antonio de prado', 'manrique', 'doce de octubre', 
        'santa cruz', 'popular',
        # Sectores específicos que aparecen en titles
        'jardines', 'niquia', 'aves marias', 'loma del escobero', 'la rochela',
        'suramérica', 'suramerica', 'estadio', 'centro', 'boston', 'manila'
    ]
    
    for ubicacion in ubicaciones_title:
        patron = rf'\b{re.escape(ubicacion)}\b'
        if re.search(patron, title_lower):
            # Mapear a nombre estándar
            if ubicacion in ['poblado', 'el poblado']:
                return 'El Poblado'
            elif ubicacion in ['belen', 'belén']:
                return 'Belén'
            elif ubicacion in ['america', 'la américa']:
                return 'La América'
            elif ubicacion in ['san cristobal', 'san cristóbal']:
                return 'San Cristóbal'
            elif ubicacion in ['suramerica', 'suramérica']:
                return 'Suramérica'
            else:
                return ubicacion.title()
    
    return None

print("✅ Funciones de extracción desde title creadas")

# Probar funciones en muestra
print(f"\n🧪 PRUEBA EN MUESTRA DE TÍTULOS:")
print("-" * 35)

muestra_titles = df_clean['title'].dropna().sample(1000, random_state=42)

ciudades_title = muestra_titles.apply(extraer_ciudad_desde_title).notna().sum()
barrios_title = muestra_titles.apply(extraer_barrio_desde_title).notna().sum()

print(f"Ciudades desde title: {ciudades_title}/1000 ({(ciudades_title/1000*100):.1f}%)")
print(f"Barrios desde title: {barrios_title}/1000 ({(barrios_title/1000*100):.1f}%)")

# Ejemplos exitosos
print(f"\n📋 EJEMPLOS EXITOSOS DESDE TITLES:")
print("-" * 40)

ejemplos_ciudad_title = muestra_titles[muestra_titles.apply(extraer_ciudad_desde_title).notna()].head(3)
ejemplos_barrio_title = muestra_titles[muestra_titles.apply(extraer_barrio_desde_title).notna()].head(3)

print("🏙️ CIUDADES DESDE TITLE:")
for i, title in enumerate(ejemplos_ciudad_title, 1):
    ciudad = extraer_ciudad_desde_title(title)
    print(f"  {i}. {ciudad} ← {str(title)[:60]}...")

print("\n🏘️ BARRIOS/SECTORES DESDE TITLE:")
for i, title in enumerate(ejemplos_barrio_title, 1):
    barrio = extraer_barrio_desde_title(title)
    print(f"  {i}. {barrio} ← {str(title)[:60]}...")

print(f"\n✅ PRUEBAS COMPLETADAS")

🔧 CREANDO FUNCIÓN DE EXTRACCIÓN DESDE TITLES
✅ Funciones de extracción desde title creadas

🧪 PRUEBA EN MUESTRA DE TÍTULOS:
-----------------------------------
Ciudades desde title: 849/1000 (84.9%)
Barrios desde title: 129/1000 (12.9%)

📋 EJEMPLOS EXITOSOS DESDE TITLES:
----------------------------------------
🏙️ CIUDADES DESDE TITLE:
  1. Medellín ← Apartamento en Venta Ubicado en MEDELLIN...
  2. Medellín ← APARTAMENTO EN ARRIENDO, MEDELLIN-LOMA DE LOS BERNAL...
  3. Medellín ← Apartamento en Arriendo Ubicado en MEDELLIN...

🏘️ BARRIOS/SECTORES DESDE TITLE:
  1. El Poblado ← SE ARRIENDA APARTAESTUDIO EN SANTA MARIA DE LOS ANGELES , PO...
  2. El Poblado ← APARTAMENTO EN ARRIENDO, MEDELLIN-POBLADO...
  3. Estadio ← Apartamento en venta Estadio 106 mt² Exito Colombia...

✅ PRUEBAS COMPLETADAS


In [46]:
# ===============================================================
# APLICACIÓN DE EXTRACCIÓN DESDE TITLES AL DATASET COMPLETO
# ===============================================================

print("🚀 EXTRACCIÓN DESDE TITLES - DATASET COMPLETO")
print("=" * 55)

# PASO 1: EXTRACCIÓN ADICIONAL DE CIUDADES DESDE TITLES
print("\n🏙️ EXTRACCIÓN ADICIONAL DE CIUDADES DESDE TITLES:")
print("-" * 55)

# Buscar registros que NO tienen ciudad pero SÍ tienen title
mask_sin_ciudad_con_title = (
    df_clean['l3_final'].isna() & 
    df_clean['title'].notna()
)

candidatos_ciudad_title = mask_sin_ciudad_con_title.sum()
print(f"Candidatos ciudad desde title: {candidatos_ciudad_title:,}")

if candidatos_ciudad_title > 0:
    # Extraer ciudades desde title
    extracciones_ciudad_title = df_clean.loc[mask_sin_ciudad_con_title, 'title'].apply(extraer_ciudad_desde_title)
    df_clean.loc[mask_sin_ciudad_con_title, 'l3_title_extracted'] = extracciones_ciudad_title
    
    ciudades_title_extraidas = df_clean['l3_title_extracted'].notna().sum()
    tasa_ciudad_title = (ciudades_title_extraidas / candidatos_ciudad_title * 100) if candidatos_ciudad_title > 0 else 0
    
    print(f"✅ Ciudades extraídas desde title: {ciudades_title_extraidas:,}")
    print(f"📈 Tasa de éxito: {tasa_ciudad_title:.1f}%")
    
    # Integrar ciudades desde title
    df_clean['l3_final'] = df_clean['l3_final'].fillna(df_clean['l3_title_extracted'])
else:
    print("No hay candidatos adicionales para ciudades desde title")
    ciudades_title_extraidas = 0

# PASO 2: EXTRACCIÓN ADICIONAL DE BARRIOS DESDE TITLES
print(f"\n🏘️ EXTRACCIÓN ADICIONAL DE BARRIOS DESDE TITLES:")
print("-" * 55)

# Buscar registros que NO tienen barrio pero SÍ tienen title
mask_sin_barrio_con_title = (
    df_clean['l4_final'].isna() & 
    df_clean['title'].notna()
)

candidatos_barrio_title = mask_sin_barrio_con_title.sum()
print(f"Candidatos barrio desde title: {candidatos_barrio_title:,}")

# Extraer barrios desde title
extracciones_barrio_title = df_clean.loc[mask_sin_barrio_con_title, 'title'].apply(extraer_barrio_desde_title)
df_clean.loc[mask_sin_barrio_con_title, 'l4_title_extracted'] = extracciones_barrio_title

barrios_title_extraidos = df_clean['l4_title_extracted'].notna().sum()
tasa_barrio_title = (barrios_title_extraidos / candidatos_barrio_title * 100) if candidatos_barrio_title > 0 else 0

print(f"✅ Barrios extraídos desde title: {barrios_title_extraidos:,}")
print(f"📈 Tasa de éxito: {tasa_barrio_title:.1f}%")

# Integrar barrios desde title
df_clean['l4_final'] = df_clean['l4_final'].fillna(df_clean['l4_title_extracted'])

# PASO 3: RESUMEN FINAL DE EXTRACCIÓN DESDE TITLES
print(f"\n📊 RESUMEN FINAL - EXTRACCIÓN DESDE TITLES:")
print("-" * 50)

print(f"🏙️ Ciudades adicionales desde title: {ciudades_title_extraidas:,}")
print(f"🏘️ Barrios adicionales desde title: {barrios_title_extraidos:,}")
print(f"🎯 Total ubicaciones desde title: {ciudades_title_extraidas + barrios_title_extraidos:,}")

# Estado final de ubicaciones
ciudades_finales = df_clean['l3_final'].notna().sum()
barrios_finales = df_clean['l4_final'].notna().sum()

print(f"\n📈 ESTADO FINAL DE UBICACIONES:")
print("-" * 35)
print(f"Ciudades finales: {ciudades_finales:,}")
print(f"Barrios finales: {barrios_finales:,}")

# Distribución de extracciones desde title
if 'l4_title_extracted' in df_clean.columns:
    print(f"\n📋 TOP BARRIOS EXTRAÍDOS DESDE TITLE:")
    print("-" * 40)
    top_barrios_title = df_clean['l4_title_extracted'].value_counts().head(10)
    for barrio, count in top_barrios_title.items():
        print(f"   {barrio:<20}: {count:>4,}")

if 'l3_title_extracted' in df_clean.columns:
    print(f"\n📋 TOP CIUDADES EXTRAÍDAS DESDE TITLE:")
    print("-" * 40)
    top_ciudades_title = df_clean['l3_title_extracted'].value_counts().head(10)
    for ciudad, count in top_ciudades_title.items():
        print(f"   {ciudad:<20}: {count:>4,}")

# Nuevo potencial de imputación de coordenadas
coords_faltantes_final = df_clean[['lat', 'lon']].isna().any(axis=1).sum()
barrios_disponibles_final = df_clean['l4_final'].notna().sum()

sin_coords_con_barrio_final = (
    df_clean[['lat', 'lon']].isna().any(axis=1) & 
    df_clean['l4_final'].notna()
).sum()

print(f"\n🗺️ IMPACTO FINAL EN COORDENADAS:")
print("-" * 35)
print(f"Coordenadas faltantes: {coords_faltantes_final:,}")
print(f"Barrios disponibles: {barrios_disponibles_final:,}")
print(f"🎯 Casos para imputar coords: {sin_coords_con_barrio_final:,}")

print(f"\n🏆 EXTRACCIÓN DESDE TITLES: COMPLETADA")
print(f"💎 Total innovación text mining: Description + Title")

🚀 EXTRACCIÓN DESDE TITLES - DATASET COMPLETO

🏙️ EXTRACCIÓN ADICIONAL DE CIUDADES DESDE TITLES:
-------------------------------------------------------
Candidatos ciudad desde title: 2,669
✅ Ciudades extraídas desde title: 322
📈 Tasa de éxito: 12.1%

🏘️ EXTRACCIÓN ADICIONAL DE BARRIOS DESDE TITLES:
-------------------------------------------------------
Candidatos barrio desde title: 186,100
✅ Barrios extraídos desde title: 6,477
📈 Tasa de éxito: 3.5%

📊 RESUMEN FINAL - EXTRACCIÓN DESDE TITLES:
--------------------------------------------------
🏙️ Ciudades adicionales desde title: 322
🏘️ Barrios adicionales desde title: 6,477
🎯 Total ubicaciones desde title: 6,799

📈 ESTADO FINAL DE UBICACIONES:
-----------------------------------
Ciudades finales: 275,508
Barrios finales: 98,232

📋 TOP BARRIOS EXTRAÍDOS DESDE TITLE:
----------------------------------------
   Suramérica          : 1,321
   El Poblado          : 1,314
   Niquia              : 1,206
   Loma Del Escobero   : 1,204
   Cen

## 🔟 **Fase 10: Imputación de Coordenadas** 🗺️

**Objetivo:** Imputar coordenadas faltantes usando centroides de barrios conocidos

**Estrategia:** Para los 25,665 casos que tienen barrio pero NO tienen coordenadas, calcular el centroide (lat/lon promedio) de cada barrio y asignar esas coordenadas.

**Justificación:** Esta estrategia permite recuperar ubicación geográfica aproximada manteniendo coherencia espacial por barrio.

In [48]:
# ===============================================================
# FASE 10: IMPUTACIÓN DE COORDENADAS POR CENTROIDE DE BARRIO
# ===============================================================

print("🗺️ FASE 10: IMPUTACIÓN DE COORDENADAS")
print("=" * 40)

# PASO 1: Identificar casos para imputación
print("🔍 IDENTIFICANDO CASOS PARA IMPUTACIÓN:")
print("-" * 40)

# Casos sin coordenadas pero CON barrio
sin_coords = df_clean[['lat', 'lon']].isna().any(axis=1)
con_barrio = df_clean['l4_final'].notna()
casos_imputar = sin_coords & con_barrio

print(f"Total registros: {len(df_clean):,}")
print(f"Sin coordenadas: {sin_coords.sum():,}")
print(f"Con barrio disponible: {con_barrio.sum():,}")
print(f"🎯 Casos a imputar: {casos_imputar.sum():,}")

# PASO 2: Calcular centroides por barrio
print(f"\n📊 CALCULANDO CENTROIDES POR BARRIO:")
print("-" * 40)

# Solo usar registros CON coordenadas para calcular centroides
con_coords = df_clean[['lat', 'lon']].notna().all(axis=1)
datos_centroides = df_clean[con_coords & df_clean['l4_final'].notna()]

print(f"Registros con coords y barrio: {len(datos_centroides):,}")

# Calcular centroide (promedio) por barrio
centroides_barrios = datos_centroides.groupby('l4_final')[['lat', 'lon']].agg({
    'lat': 'mean',
    'lon': 'mean'
}).round(6)

print(f"Barrios con centroide calculado: {len(centroides_barrios):,}")

# Mostrar algunos ejemplos de centroides
print(f"\n📋 EJEMPLOS DE CENTROIDES CALCULADOS:")
print("-" * 45)
for barrio, coords in centroides_barrios.head(8).iterrows():
    print(f"   {barrio:<25}: ({coords['lat']:.6f}, {coords['lon']:.6f})")

print(f"\n✅ Centroides calculados para {len(centroides_barrios):,} barrios")

🗺️ FASE 10: IMPUTACIÓN DE COORDENADAS
🔍 IDENTIFICANDO CASOS PARA IMPUTACIÓN:
----------------------------------------
Total registros: 277,856
Sin coordenadas: 163,274
Con barrio disponible: 98,232
🎯 Casos a imputar: 25,665

📊 CALCULANDO CENTROIDES POR BARRIO:
----------------------------------------
Registros con coords y barrio: 72,567
Barrios con centroide calculado: 32

📋 EJEMPLOS DE CENTROIDES CALCULADOS:
---------------------------------------------
   Altavista                : (6.216012, -75.609753)
   Aranjuez                 : (6.252886, -75.564322)
   Aves Marias              : (6.147722, -75.614887)
   Belén                    : (6.222420, -75.598307)
   Boston                   : (6.169000, -75.654000)
   Buenos Aires             : (6.231760, -75.557261)
   Candelaria               : (6.246639, -75.564643)
   Castilla                 : (6.217384, -75.504957)

✅ Centroides calculados para 32 barrios


In [49]:
# ===============================================================
# APLICAR IMPUTACIÓN DE COORDENADAS
# ===============================================================

print("🚀 APLICANDO IMPUTACIÓN DE COORDENADAS:")
print("=" * 45)

# PASO 3: Aplicar imputación
imputaciones_realizadas = 0

# Crear copias de las columnas para imputación
df_clean['lat_imputada'] = df_clean['lat'].copy()
df_clean['lon_imputada'] = df_clean['lon'].copy()

# Para cada registro que necesita imputación
for idx in df_clean[casos_imputar].index:
    barrio = df_clean.loc[idx, 'l4_final']
    
    # Si tenemos centroide para este barrio
    if barrio in centroides_barrios.index:
        # Imputar coordenadas con el centroide del barrio
        df_clean.loc[idx, 'lat_imputada'] = centroides_barrios.loc[barrio, 'lat']
        df_clean.loc[idx, 'lon_imputada'] = centroides_barrios.loc[barrio, 'lon']
        imputaciones_realizadas += 1

print(f"✅ Imputaciones realizadas: {imputaciones_realizadas:,}")

# PASO 4: Verificar resultados
print(f"\n📊 VERIFICACIÓN DE RESULTADOS:")
print("-" * 35)

# Antes vs después
coords_faltantes_antes = df_clean[['lat', 'lon']].isna().any(axis=1).sum()
coords_faltantes_despues = df_clean[['lat_imputada', 'lon_imputada']].isna().any(axis=1).sum()
coords_completadas = coords_faltantes_antes - coords_faltantes_despues

print(f"Coordenadas faltantes antes: {coords_faltantes_antes:,}")
print(f"Coordenadas faltantes después: {coords_faltantes_despues:,}")
print(f"🎯 Coordenadas completadas: {coords_completadas:,}")

if coords_faltantes_antes > 0:
    mejora_porcentual = (coords_completadas / coords_faltantes_antes) * 100
    print(f"📈 Mejora: {mejora_porcentual:.1f}%")

# Cobertura total de coordenadas
cobertura_total = ((len(df_clean) - coords_faltantes_despues) / len(df_clean)) * 100
print(f"🗺️ Cobertura total de coordenadas: {cobertura_total:.1f}%")

print(f"\n🏆 FASE 10 COMPLETADA - IMPUTACIÓN DE COORDENADAS")
print(f"💎 {imputaciones_realizadas:,} coordenadas imputadas por centroide de barrio")

🚀 APLICANDO IMPUTACIÓN DE COORDENADAS:
✅ Imputaciones realizadas: 25,620

📊 VERIFICACIÓN DE RESULTADOS:
-----------------------------------
Coordenadas faltantes antes: 163,274
Coordenadas faltantes después: 137,654
🎯 Coordenadas completadas: 25,620
📈 Mejora: 15.7%
🗺️ Cobertura total de coordenadas: 50.5%

🏆 FASE 10 COMPLETADA - IMPUTACIÓN DE COORDENADAS
💎 25,620 coordenadas imputadas por centroide de barrio
✅ Imputaciones realizadas: 25,620

📊 VERIFICACIÓN DE RESULTADOS:
-----------------------------------
Coordenadas faltantes antes: 163,274
Coordenadas faltantes después: 137,654
🎯 Coordenadas completadas: 25,620
📈 Mejora: 15.7%
🗺️ Cobertura total de coordenadas: 50.5%

🏆 FASE 10 COMPLETADA - IMPUTACIÓN DE COORDENADAS
💎 25,620 coordenadas imputadas por centroide de barrio


In [50]:
# ===============================================================
# ANÁLISIS DE CALIDAD DE IMPUTACIÓN
# ===============================================================

print("🔍 ANÁLISIS DE CALIDAD DE IMPUTACIÓN:")
print("=" * 45)

# PASO 5: Mostrar ejemplos de imputación por barrio
print("📋 EJEMPLOS DE IMPUTACIÓN POR BARRIO:")
print("-" * 40)

# Contar imputaciones por barrio
imputaciones_por_barrio = df_clean[casos_imputar]['l4_final'].value_counts().head(10)

print("Los 10 barrios con más coordenadas imputadas:")
for barrio, count in imputaciones_por_barrio.items():
    if barrio in centroides_barrios.index:
        lat_centroide = centroides_barrios.loc[barrio, 'lat']
        lon_centroide = centroides_barrios.loc[barrio, 'lon']
        print(f"   {barrio:<25}: {count:>3,} imputaciones → ({lat_centroide:.6f}, {lon_centroide:.6f})")

# PASO 6: Validación geográfica
print(f"\n🌍 VALIDACIÓN GEOGRÁFICA:")
print("-" * 30)

# Verificar que coordenadas imputadas están en rangos válidos de Colombia
LAT_MIN, LAT_MAX = -4.23, 15.52
LON_MIN, LON_MAX = -79.01, -66.85

coords_imputadas_invalidas = df_clean[
    df_clean['lat_imputada'].notna() & df_clean['lon_imputada'].notna()
].apply(lambda row: (
    row['lat_imputada'] < LAT_MIN or row['lat_imputada'] > LAT_MAX or
    row['lon_imputada'] < LON_MIN or row['lon_imputada'] > LON_MAX
), axis=1).sum()

print(f"Coordenadas imputadas fuera de Colombia: {coords_imputadas_invalidas:,}")
print(f"✅ Calidad geográfica: {'EXCELENTE' if coords_imputadas_invalidas == 0 else 'REVISAR'}")

# PASO 7: Resumen final de coordenadas
print(f"\n📊 RESUMEN FINAL DE COORDENADAS:")
print("-" * 35)

# Clasificar registros por origen de coordenadas
coords_originales = df_clean[['lat', 'lon']].notna().all(axis=1).sum()
coords_solo_imputadas = (
    df_clean[['lat', 'lon']].isna().any(axis=1) & 
    df_clean[['lat_imputada', 'lon_imputada']].notna().all(axis=1)
).sum()
coords_sin_completar = df_clean[['lat_imputada', 'lon_imputada']].isna().any(axis=1).sum()

print(f"Coordenadas originales: {coords_originales:,}")
print(f"Coordenadas imputadas: {coords_solo_imputadas:,}")
print(f"Sin coordenadas: {coords_sin_completar:,}")
print(f"Total con ubicación: {coords_originales + coords_solo_imputadas:,}")

print(f"\n✅ ANÁLISIS DE CALIDAD COMPLETADO")
print(f"🎯 Dataset listo para análisis geoespacial")

🔍 ANÁLISIS DE CALIDAD DE IMPUTACIÓN:
📋 EJEMPLOS DE IMPUTACIÓN POR BARRIO:
----------------------------------------
Los 10 barrios con más coordenadas imputadas:
   El Poblado               : 8,022 imputaciones → (6.201595, -75.568125)
   Belén                    : 5,822 imputaciones → (6.222420, -75.598307)
   Laureles                 : 4,072 imputaciones → (6.243746, -75.591531)
   Robledo                  : 3,555 imputaciones → (6.276740, -75.592606)
   La América               : 1,756 imputaciones → (6.255715, -75.603495)
   San Javier               : 517 imputaciones → (6.259451, -75.611224)
   Aranjuez                 : 385 imputaciones → (6.252886, -75.564322)
   Suramérica               : 242 imputaciones → (6.179759, -75.606737)
   Guayabal                 : 215 imputaciones → (6.207271, -75.591987)
   Manrique                 : 170 imputaciones → (6.269850, -75.549730)

🌍 VALIDACIÓN GEOGRÁFICA:
------------------------------
Coordenadas imputadas fuera de Colombia: 0
✅ Calidad

## 1️⃣1️⃣ **Fase 11: Modelado Avanzado de Superficie** 📐

**Objetivo:** Desarrollar un modelo de Machine Learning para imputar `surface_total` faltante usando variables disponibles

**Estrategia:** 
1. Usar variables predictoras: bedrooms, bathrooms, rooms, price, ubicación
2. Entrenar modelo ML (Random Forest/XGBoost) con registros completos
3. Predecir superficie en registros sin `surface_total`

**Justificación:** La superficie es crucial para predicción de precios. Un modelo ML puede capturar patrones complejos entre habitaciones, precio y ubicación para estimar superficie de manera más precisa que reglas simples.

In [52]:
# ===============================================================
# FASE 11: ANÁLISIS INICIAL PARA MODELADO DE SUPERFICIE
# ===============================================================

print("📐 FASE 11: MODELADO AVANZADO DE SUPERFICIE")
print("=" * 50)

# PASO 1: Analizar estado actual de surface_total_final
print("🔍 ANÁLISIS DE SUPERFICIE ACTUAL:")
print("-" * 35)

superficie_disponible = df_clean['surface_total_final'].notna().sum()
superficie_faltante = df_clean['surface_total_final'].isna().sum()
total = len(df_clean)

print(f"Total registros: {total:,}")
print(f"Con superficie: {superficie_disponible:,} ({(superficie_disponible/total*100):.1f}%)")
print(f"Sin superficie: {superficie_faltante:,} ({(superficie_faltante/total*100):.1f}%)")

# PASO 2: Analizar variables predictoras disponibles
print(f"\n📊 VARIABLES PREDICTORAS DISPONIBLES:")
print("-" * 40)

variables_predictoras = ['bedrooms_final', 'bathrooms_final', 'rooms_final', 'price', 'l3_final', 'l4_final']

for var in variables_predictoras:
    if var in df_clean.columns:
        disponible = df_clean[var].notna().sum()
        faltante = df_clean[var].isna().sum()
        pct = (disponible/total*100)
        print(f"{var:<20}: {disponible:>8,} ({pct:>5.1f}%) disponible")

# PASO 3: Identificar registros útiles para entrenamiento
print(f"\n🎯 REGISTROS PARA ENTRENAMIENTO:")
print("-" * 35)

# Registros CON superficie Y con al menos algunas variables predictoras
con_superficie = df_clean['surface_total_final'].notna()
con_bedrooms = df_clean['bedrooms_final'].notna()
con_price = df_clean['price'].notna()  # precio siempre disponible
con_ubicacion = df_clean['l3_final'].notna()

registros_entrenamiento = (con_superficie & con_bedrooms & con_price & con_ubicacion).sum()
print(f"Registros completos para entrenamiento: {registros_entrenamiento:,}")

# Registros SIN superficie pero CON variables predictoras
sin_superficie = df_clean['surface_total_final'].isna()
registros_prediccion = (sin_superficie & con_bedrooms & con_price & con_ubicacion).sum()
print(f"Registros para predicción: {registros_prediccion:,}")

print(f"\n✅ ANÁLISIS INICIAL COMPLETADO")
print(f"🎯 Casos viables para modelado: {registros_entrenamiento:,} entrenamiento + {registros_prediccion:,} predicción")

📐 FASE 11: MODELADO AVANZADO DE SUPERFICIE
🔍 ANÁLISIS DE SUPERFICIE ACTUAL:
-----------------------------------
Total registros: 277,856
Con superficie: 72,377 (26.0%)
Sin superficie: 205,479 (74.0%)

📊 VARIABLES PREDICTORAS DISPONIBLES:
----------------------------------------
bedrooms_final      :  216,320 ( 77.9%) disponible
bathrooms_final     :  265,233 ( 95.5%) disponible
rooms_final         :  124,037 ( 44.6%) disponible
price               :  277,856 (100.0%) disponible
l3_final            :  275,508 ( 99.2%) disponible
l4_final            :   98,232 ( 35.4%) disponible

🎯 REGISTROS PARA ENTRENAMIENTO:
-----------------------------------
Registros completos para entrenamiento: 60,053
Registros para predicción: 154,258

✅ ANÁLISIS INICIAL COMPLETADO
🎯 Casos viables para modelado: 60,053 entrenamiento + 154,258 predicción


In [53]:
# ===============================================================
# PREPARACIÓN DE DATOS PARA MODELO DE SUPERFICIE
# ===============================================================

print("🔧 PREPARACIÓN DE DATOS PARA MODELO ML")
print("=" * 45)

# Importar librerías de ML
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.preprocessing import LabelEncoder
import numpy as np

print("✅ Librerías ML importadas")

# PASO 1: Crear dataset de entrenamiento
print(f"\n📊 CREANDO DATASET DE ENTRENAMIENTO:")
print("-" * 40)

# Filtro para registros de entrenamiento (CON superficie Y variables clave)
mask_entrenamiento = (
    df_clean['surface_total_final'].notna() &
    df_clean['bedrooms_final'].notna() &
    df_clean['price'].notna() &
    df_clean['l3_final'].notna()
)

df_train = df_clean[mask_entrenamiento].copy()
print(f"Registros de entrenamiento: {len(df_train):,}")

# PASO 2: Crear variables predictoras
print(f"\n🎯 CREANDO VARIABLES PREDICTORAS:")
print("-" * 35)

# Variables numéricas
variables_numericas = ['bedrooms_final', 'bathrooms_final', 'price']

# Codificar variables categóricas (ciudad)
le_ciudad = LabelEncoder()
df_train['ciudad_encoded'] = le_ciudad.fit_transform(df_train['l3_final'].fillna('Unknown'))

# Variable objetivo
y_train = df_train['surface_total_final']

# Variables predictoras
X_features = []
feature_names = []

# Agregar variables numéricas
for var in variables_numericas:
    if var in df_train.columns:
        X_features.append(df_train[var].fillna(0))  # Llenar NaN con 0
        feature_names.append(var)

# Agregar ciudad codificada
X_features.append(df_train['ciudad_encoded'])
feature_names.append('ciudad')

# Agregar rooms si está disponible
if 'rooms_final' in df_train.columns:
    X_features.append(df_train['rooms_final'].fillna(0))
    feature_names.append('rooms_final')

# Crear matriz X
import pandas as pd
X_train = pd.DataFrame(dict(zip(feature_names, X_features)))

print(f"Variables predictoras: {feature_names}")
print(f"Forma del dataset: X={X_train.shape}, y={y_train.shape}")

# Estadísticas de la variable objetivo
print(f"\n📐 ESTADÍSTICAS DE SUPERFICIE (objetivo):")
print("-" * 40)
print(f"Media: {y_train.mean():.1f} m²")
print(f"Mediana: {y_train.median():.1f} m²")
print(f"Mín: {y_train.min():.1f} m²")
print(f"Máx: {y_train.max():.1f} m²")

print(f"\n✅ DATOS PREPARADOS PARA ENTRENAMIENTO")

🔧 PREPARACIÓN DE DATOS PARA MODELO ML
✅ Librerías ML importadas

📊 CREANDO DATASET DE ENTRENAMIENTO:
----------------------------------------
Registros de entrenamiento: 60,053

🎯 CREANDO VARIABLES PREDICTORAS:
-----------------------------------
Variables predictoras: ['bedrooms_final', 'bathrooms_final', 'price', 'ciudad', 'rooms_final']
Forma del dataset: X=(60053, 5), y=(60053,)

📐 ESTADÍSTICAS DE SUPERFICIE (objetivo):
----------------------------------------
Media: 122.0 m²
Mediana: 80.0 m²
Mín: 10.0 m²
Máx: 83333.0 m²

✅ DATOS PREPARADOS PARA ENTRENAMIENTO


In [54]:
# ===============================================================
# ENTRENAMIENTO DEL MODELO DE SUPERFICIE
# ===============================================================

print("🚀 ENTRENANDO MODELO DE SUPERFICIE")
print("=" * 40)

# PASO 1: Dividir datos en entrenamiento y validación
X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42
)

print(f"División de datos:")
print(f"  Entrenamiento: {X_train_split.shape[0]:,} registros")
print(f"  Validación: {X_val_split.shape[0]:,} registros")

# PASO 2: Entrenar modelo Random Forest
print(f"\n🌳 ENTRENANDO RANDOM FOREST:")
print("-" * 30)

# Configurar modelo (parámetros conservadores para rapidez)
rf_model = RandomForestRegressor(
    n_estimators=100,           # Número de árboles
    max_depth=10,              # Profundidad máxima
    min_samples_split=5,       # Mín. muestras para dividir
    min_samples_leaf=2,        # Mín. muestras por hoja
    random_state=42,
    n_jobs=-1                  # Usar todos los cores disponibles
)

# Entrenar modelo
print("Entrenando...")
rf_model.fit(X_train_split, y_train_split)
print("✅ Modelo entrenado")

# PASO 3: Evaluar modelo
print(f"\n📊 EVALUACIÓN DEL MODELO:")
print("-" * 30)

# Predicciones en conjunto de validación
y_pred = rf_model.predict(X_val_split)

# Métricas
mae = mean_absolute_error(y_val_split, y_pred)
r2 = r2_score(y_val_split, y_pred)

print(f"Error Absoluto Medio (MAE): {mae:.1f} m²")
print(f"R² Score: {r2:.3f}")

# Interpretación de las métricas
if r2 > 0.7:
    calidad = "EXCELENTE"
elif r2 > 0.5:
    calidad = "BUENA"
elif r2 > 0.3:
    calidad = "MODERADA"
else:
    calidad = "BAJA"

print(f"Calidad del modelo: {calidad}")

# PASO 4: Importancia de variables
print(f"\n🎯 IMPORTANCIA DE VARIABLES:")
print("-" * 30)

importancias = rf_model.feature_importances_
for i, (feature, importance) in enumerate(zip(feature_names, importancias)):
    print(f"  {feature:<15}: {importance:.3f}")

print(f"\n✅ MODELO LISTO PARA PREDICCIÓN")

🚀 ENTRENANDO MODELO DE SUPERFICIE
División de datos:
  Entrenamiento: 48,042 registros
  Validación: 12,011 registros

🌳 ENTRENANDO RANDOM FOREST:
------------------------------
Entrenando...
✅ Modelo entrenado

📊 EVALUACIÓN DEL MODELO:
------------------------------
Error Absoluto Medio (MAE): 31.0 m²
R² Score: 0.313
Calidad del modelo: MODERADA

🎯 IMPORTANCIA DE VARIABLES:
------------------------------
  bedrooms_final : 0.035
  bathrooms_final: 0.132
  price          : 0.495
  ciudad         : 0.221
  rooms_final    : 0.117

✅ MODELO LISTO PARA PREDICCIÓN
✅ Modelo entrenado

📊 EVALUACIÓN DEL MODELO:
------------------------------
Error Absoluto Medio (MAE): 31.0 m²
R² Score: 0.313
Calidad del modelo: MODERADA

🎯 IMPORTANCIA DE VARIABLES:
------------------------------
  bedrooms_final : 0.035
  bathrooms_final: 0.132
  price          : 0.495
  ciudad         : 0.221
  rooms_final    : 0.117

✅ MODELO LISTO PARA PREDICCIÓN


## 🧠 **Optimización del Modelo ML como Experto**

**Problemas Detectados en el Modelo Actual:**
1. **Outliers extremos**: Superficie máxima de 83,333 m² contamina el entrenamiento
2. **Variables categóricas suboptimizadas**: Solo usamos ciudad, falta barrio (más granular)
3. **Ingeniería de features faltante**: No aprovechamos relaciones como precio/m²
4. **Parámetros conservadores**: Random Forest muy limitado (solo 100 árboles, profundidad 10)
5. **Datos de entrenamiento limitados**: Podríamos usar más variables disponibles

**Estrategia de Mejora:**
1. Limpieza agresiva de outliers
2. Mejores features de ubicación y engineered features
3. Optimización de hiperparámetros
4. Validación cruzada para evaluar estabilidad

In [55]:
# ===============================================================
# ANÁLISIS EXPLORATORIO COMO EXPERTO ML
# ===============================================================

print("🧠 ANÁLISIS COMO EXPERTO EN MACHINE LEARNING")
print("=" * 50)

# PASO 1: Análisis de outliers en superficie
print("🔍 ANÁLISIS DE OUTLIERS EN SUPERFICIE:")
print("-" * 40)

# Estadísticas detalladas de superficie
superficie_stats = df_train['surface_total_final'].describe()
print("Estadísticas de superficie:")
print(superficie_stats)

# Detectar outliers usando IQR
Q1 = df_train['surface_total_final'].quantile(0.25)
Q3 = df_train['surface_total_final'].quantile(0.75)
IQR = Q3 - Q1
limite_inferior = Q1 - 1.5 * IQR
limite_superior = Q3 + 1.5 * IQR

outliers_extremos = (df_train['surface_total_final'] > limite_superior).sum()
print(f"\nDetección de outliers (IQR):")
print(f"Q1: {Q1:.1f} m²")
print(f"Q3: {Q3:.1f} m²")
print(f"Límite superior (Q3 + 1.5*IQR): {limite_superior:.1f} m²")
print(f"Outliers extremos: {outliers_extremos:,} ({(outliers_extremos/len(df_train)*100):.1f}%)")

# PASO 2: Análisis de correlaciones
print(f"\n📊 ANÁLISIS DE CORRELACIONES:")
print("-" * 35)

# Calcular correlaciones con superficie
vars_numericas = ['bedrooms_final', 'bathrooms_final', 'rooms_final', 'price']
correlaciones = {}

for var in vars_numericas:
    if var in df_train.columns:
        # Filtrar valores no nulos para ambas variables
        mask_validos = df_train[var].notna() & df_train['surface_total_final'].notna()
        if mask_validos.sum() > 100:  # Mínimo 100 casos para correlación confiable
            corr = df_train[mask_validos][var].corr(df_train[mask_validos]['surface_total_final'])
            correlaciones[var] = corr
            print(f"{var:<15}: {corr:>6.3f}")

# PASO 3: Análisis de variables categóricas
print(f"\n🏙️ ANÁLISIS DE UBICACIÓN:")
print("-" * 30)

# Superficie promedio por ciudad
superficie_por_ciudad = df_train.groupby('l3_final')['surface_total_final'].agg(['mean', 'count']).round(1)
superficie_por_ciudad = superficie_por_ciudad[superficie_por_ciudad['count'] >= 50]  # Mín. 50 casos
superficie_por_ciudad = superficie_por_ciudad.sort_values('mean', ascending=False)

print("Superficie promedio por ciudad (top 10):")
for ciudad, stats in superficie_por_ciudad.head(10).iterrows():
    print(f"  {ciudad:<20}: {stats['mean']:>6.1f} m² ({int(stats['count']):>4,} casos)")

# PASO 4: Potencial de barrios
barrios_con_datos = df_train[df_train['l4_final'].notna()]
if len(barrios_con_datos) > 1000:
    print(f"\n🏘️ POTENCIAL DE BARRIOS:")
    print("-" * 25)
    superficie_por_barrio = barrios_con_datos.groupby('l4_final')['surface_total_final'].agg(['mean', 'count']).round(1)
    superficie_por_barrio = superficie_por_barrio[superficie_por_barrio['count'] >= 20]  # Mín. 20 casos
    superficie_por_barrio = superficie_por_barrio.sort_values('mean', ascending=False)
    
    print(f"Barrios con datos suficientes: {len(superficie_por_barrio):,}")
    print("Superficie promedio por barrio (top 8):")
    for barrio, stats in superficie_por_barrio.head(8).iterrows():
        print(f"  {barrio:<20}: {stats['mean']:>6.1f} m² ({int(stats['count']):>3,} casos)")

print(f"\n✅ ANÁLISIS EXPLORATORIO COMPLETADO")
print(f"🎯 Identificados múltiples vectores de mejora")

🧠 ANÁLISIS COMO EXPERTO EN MACHINE LEARNING
🔍 ANÁLISIS DE OUTLIERS EN SUPERFICIE:
----------------------------------------
Estadísticas de superficie:
count    60053.000000
mean       122.003893
std        422.704869
min         10.000000
25%         65.000000
50%         80.000000
75%        120.000000
max      83333.000000
Name: surface_total_final, dtype: float64

Detección de outliers (IQR):
Q1: 65.0 m²
Q3: 120.0 m²
Límite superior (Q3 + 1.5*IQR): 202.5 m²
Outliers extremos: 5,476 (9.1%)

📊 ANÁLISIS DE CORRELACIONES:
-----------------------------------
bedrooms_final :  0.130
bathrooms_final:  0.181
rooms_final    :  0.117
price          :  0.046

🏙️ ANÁLISIS DE UBICACIÓN:
------------------------------
Superficie promedio por ciudad (top 10):
  Retiro              : 1251.3 m² (  82 casos)
  Rionegro            :  210.2 m² (1,529 casos)
  Marinilla           :  203.8 m² ( 108 casos)
  La Ceja             :  187.7 m² ( 184 casos)
  Barbosa             :  168.3 m² (  88 casos)
  Envi

In [56]:
# ===============================================================
# MODELO MEJORADO V2: APLICANDO EXPERTISE ML
# ===============================================================

print("🚀 IMPLEMENTANDO MODELO MEJORADO V2")
print("=" * 45)

# MEJORA 1: Limpieza agresiva de outliers
print("🧹 MEJORA 1: LIMPIEZA DE OUTLIERS:")
print("-" * 35)

# Filtrar outliers extremos (conservar solo datos realistas)
superficie_limite = 300  # Límite más estricto para viviendas residenciales
df_train_clean = df_train[df_train['surface_total_final'] <= superficie_limite].copy()

print(f"Registros antes: {len(df_train):,}")
print(f"Registros después: {len(df_train_clean):,}")
print(f"Outliers removidos: {len(df_train) - len(df_train_clean):,}")

# MEJORA 2: Feature Engineering avanzado
print(f"\n⚙️ MEJORA 2: FEATURE ENGINEERING AVANZADO:")
print("-" * 45)

# Feature 1: Precio por m² estimado (usando median por ciudad)
precio_por_ciudad = df_train_clean.groupby('l3_final')['price'].median()
df_train_clean['precio_mediano_ciudad'] = df_train_clean['l3_final'].map(precio_por_ciudad)
df_train_clean['precio_relativo'] = df_train_clean['price'] / df_train_clean['precio_mediano_ciudad']

# Feature 2: Densidad de habitaciones (rooms por bedroom ratio)
df_train_clean['densidad_habitaciones'] = df_train_clean['rooms_final'] / (df_train_clean['bedrooms_final'] + 1)

# Feature 3: Indicador de lujo (alta relación baños/habitaciones)
df_train_clean['ratio_banos_habitaciones'] = df_train_clean['bathrooms_final'] / (df_train_clean['bedrooms_final'] + 1)

# Feature 4: Codificación de barrio (cuando disponible)
df_train_clean['tiene_barrio'] = df_train_clean['l4_final'].notna().astype(int)

# Codificar barrios con datos suficientes
barrios_frecuentes = df_train_clean['l4_final'].value_counts()
barrios_validos = barrios_frecuentes[barrios_frecuentes >= 20].index
df_train_clean['barrio_valido'] = df_train_clean['l4_final'].apply(
    lambda x: x if x in barrios_validos else 'Otro'
)

# Encoder para barrios
le_barrio = LabelEncoder()
df_train_clean['barrio_encoded'] = le_barrio.fit_transform(df_train_clean['barrio_valido'].fillna('Sin_Barrio'))

print(f"✅ Features creados:")
print(f"  • Precio relativo por ciudad")
print(f"  • Densidad de habitaciones")
print(f"  • Ratio baños/habitaciones")
print(f"  • Barrios válidos: {len(barrios_validos):,}")

# MEJORA 3: Variables predictoras optimizadas
print(f"\n🎯 MEJORA 3: VARIABLES OPTIMIZADAS:")
print("-" * 35)

# Nuevas variables predictoras
variables_v2 = [
    'bedrooms_final', 'bathrooms_final', 'rooms_final',
    'precio_relativo', 'densidad_habitaciones', 'ratio_banos_habitaciones',
    'ciudad_encoded', 'barrio_encoded', 'tiene_barrio'
]

# Recodificar ciudad con dataset limpio
le_ciudad_v2 = LabelEncoder()
df_train_clean['ciudad_encoded'] = le_ciudad_v2.fit_transform(df_train_clean['l3_final'])

# Crear matriz X optimizada
X_v2_features = []
for var in variables_v2:
    if var in df_train_clean.columns:
        X_v2_features.append(df_train_clean[var].fillna(0))

X_train_v2 = pd.DataFrame(dict(zip(variables_v2, X_v2_features)))
y_train_v2 = df_train_clean['surface_total_final']

print(f"Variables V2: {variables_v2}")
print(f"Forma dataset V2: X={X_train_v2.shape}, y={y_train_v2.shape}")

# Verificar correlaciones mejoradas
print(f"\n📊 CORRELACIONES V2:")
print("-" * 20)
for var in ['precio_relativo', 'ratio_banos_habitaciones', 'densidad_habitaciones']:
    if var in df_train_clean.columns:
        corr = df_train_clean[var].corr(y_train_v2)
        print(f"{var:<25}: {corr:>6.3f}")

print(f"\n✅ DATASET V2 PREPARADO")

🚀 IMPLEMENTANDO MODELO MEJORADO V2
🧹 MEJORA 1: LIMPIEZA DE OUTLIERS:
-----------------------------------
Registros antes: 60,053
Registros después: 57,527
Outliers removidos: 2,526

⚙️ MEJORA 2: FEATURE ENGINEERING AVANZADO:
---------------------------------------------
✅ Features creados:
  • Precio relativo por ciudad
  • Densidad de habitaciones
  • Ratio baños/habitaciones
  • Barrios válidos: 26

🎯 MEJORA 3: VARIABLES OPTIMIZADAS:
-----------------------------------
Variables V2: ['bedrooms_final', 'bathrooms_final', 'rooms_final', 'precio_relativo', 'densidad_habitaciones', 'ratio_banos_habitaciones', 'ciudad_encoded', 'barrio_encoded', 'tiene_barrio']
Forma dataset V2: X=(57527, 9), y=(57527,)

📊 CORRELACIONES V2:
--------------------
precio_relativo          :  0.027
ratio_banos_habitaciones :  0.414
densidad_habitaciones    :  0.300

✅ DATASET V2 PREPARADO


In [57]:
# ===============================================================
# ENTRENAMIENTO MODELO OPTIMIZADO V2
# ===============================================================

print("🚀 ENTRENAMIENTO MODELO OPTIMIZADO V2")
print("=" * 45)

# División de datos optimizada
X_train_v2_split, X_val_v2_split, y_train_v2_split, y_val_v2_split = train_test_split(
    X_train_v2, y_train_v2, test_size=0.2, random_state=42
)

print(f"División V2:")
print(f"  Entrenamiento: {X_train_v2_split.shape[0]:,} registros")
print(f"  Validación: {X_val_v2_split.shape[0]:,} registros")

# MEJORA 4: Hiperparámetros optimizados
print(f"\n🎛️ MEJORA 4: HIPERPARÁMETROS OPTIMIZADOS:")
print("-" * 40)

# Modelo Random Forest optimizado
rf_model_v2 = RandomForestRegressor(
    n_estimators=200,          # Más árboles para mejor generalización
    max_depth=15,              # Más profundidad para capturar patrones complejos
    min_samples_split=10,      # Balance overfitting vs underfitting
    min_samples_leaf=5,        # Hojas más conservadoras
    max_features='sqrt',       # Feature selection automática
    bootstrap=True,            # Bagging para reducir varianza
    random_state=42,
    n_jobs=-1
)

print("Parámetros optimizados:")
print(f"  • n_estimators: 200 (vs 100 anterior)")
print(f"  • max_depth: 15 (vs 10 anterior)")
print(f"  • max_features: sqrt")
print(f"  • Variables: 9 (vs 5 anterior)")

# Entrenar modelo V2
print(f"\n🌳 ENTRENANDO RANDOM FOREST V2:")
print("-" * 35)

print("Entrenando modelo optimizado...")
rf_model_v2.fit(X_train_v2_split, y_train_v2_split)
print("✅ Modelo V2 entrenado")

# Evaluación modelo V2
print(f"\n📊 EVALUACIÓN MODELO V2:")
print("-" * 30)

y_pred_v2 = rf_model_v2.predict(X_val_v2_split)

mae_v2 = mean_absolute_error(y_val_v2_split, y_pred_v2)
r2_v2 = r2_score(y_val_v2_split, y_pred_v2)

print(f"Error Absoluto Medio (MAE) V2: {mae_v2:.1f} m²")
print(f"R² Score V2: {r2_v2:.3f}")

# Calidad V2
if r2_v2 > 0.7:
    calidad_v2 = "EXCELENTE"
elif r2_v2 > 0.5:
    calidad_v2 = "BUENA"
elif r2_v2 > 0.3:
    calidad_v2 = "MODERADA"
else:
    calidad_v2 = "BAJA"

print(f"Calidad modelo V2: {calidad_v2}")

# Comparación V1 vs V2
print(f"\n🔄 COMPARACIÓN V1 vs V2:")
print("-" * 30)
print(f"{'Métrica':<15} {'V1 (Original)':<15} {'V2 (Optimizado)':<15} {'Mejora':<10}")
print("-" * 65)
print(f"{'R² Score':<15} {r2:<15.3f} {r2_v2:<15.3f} {((r2_v2-r2)/r2*100):+6.1f}%")
print(f"{'MAE (m²)':<15} {mae:<15.1f} {mae_v2:<15.1f} {((mae-mae_v2)/mae*100):+6.1f}%")

# Importancia de features V2
print(f"\n🎯 IMPORTANCIA DE FEATURES V2:")
print("-" * 35)

importancias_v2 = rf_model_v2.feature_importances_
feature_importance_v2 = list(zip(variables_v2, importancias_v2))
feature_importance_v2.sort(key=lambda x: x[1], reverse=True)

for feature, importance in feature_importance_v2:
    print(f"  {feature:<25}: {importance:.3f}")

print(f"\n🏆 MODELO V2: {'ÉXITO' if r2_v2 > 0.5 else 'NECESITA MÁS MEJORAS'}")
print(f"💎 Mejora de performance: {((r2_v2-r2)/r2*100):+.1f}%")

🚀 ENTRENAMIENTO MODELO OPTIMIZADO V2
División V2:
  Entrenamiento: 46,021 registros
  Validación: 11,506 registros

🎛️ MEJORA 4: HIPERPARÁMETROS OPTIMIZADOS:
----------------------------------------
Parámetros optimizados:
  • n_estimators: 200 (vs 100 anterior)
  • max_depth: 15 (vs 10 anterior)
  • max_features: sqrt
  • Variables: 9 (vs 5 anterior)

🌳 ENTRENANDO RANDOM FOREST V2:
-----------------------------------
Entrenando modelo optimizado...
✅ Modelo V2 entrenado

📊 EVALUACIÓN MODELO V2:
------------------------------
Error Absoluto Medio (MAE) V2: 12.8 m²
R² Score V2: 0.807
Calidad modelo V2: EXCELENTE

🔄 COMPARACIÓN V1 vs V2:
------------------------------
Métrica         V1 (Original)   V2 (Optimizado) Mejora    
-----------------------------------------------------------------
R² Score        0.313           0.807           +157.4%
MAE (m²)        31.0            12.8             +58.7%

🎯 IMPORTANCIA DE FEATURES V2:
-----------------------------------
  bathrooms_final    

In [58]:
# ===============================================================
# APLICACIÓN DEL MODELO V2 OPTIMIZADO PARA PREDICCIONES
# ===============================================================

print("🎯 APLICANDO MODELO V2 OPTIMIZADO")
print("=" * 45)

# PASO 1: Identificar registros para predicción
print("🔍 IDENTIFICANDO REGISTROS PARA PREDICCIÓN:")
print("-" * 45)

# Registros SIN superficie pero CON variables predictoras necesarias
mask_prediccion_v2 = (
    df_clean['surface_total_final'].isna() &
    df_clean['bedrooms_final'].notna() &
    df_clean['bathrooms_final'].notna() &
    df_clean['price'].notna() &
    df_clean['l3_final'].notna()
)

df_predict_v2 = df_clean[mask_prediccion_v2].copy()
print(f"Registros elegibles para predicción: {len(df_predict_v2):,}")

# PASO 2: Preparar features V2 para predicción
print(f"\n🔧 PREPARANDO FEATURES V2 PARA PREDICCIÓN:")
print("-" * 45)

# Feature Engineering para predicción (igual que entrenamiento)
df_predict_v2['precio_mediano_ciudad'] = df_predict_v2['l3_final'].map(precio_por_ciudad)
df_predict_v2['precio_relativo'] = df_predict_v2['price'] / df_predict_v2['precio_mediano_ciudad']
df_predict_v2['densidad_habitaciones'] = df_predict_v2['rooms_final'] / (df_predict_v2['bedrooms_final'] + 1)
df_predict_v2['ratio_banos_habitaciones'] = df_predict_v2['bathrooms_final'] / (df_predict_v2['bedrooms_final'] + 1)
df_predict_v2['tiene_barrio'] = df_predict_v2['l4_final'].notna().astype(int)

# Barrios válidos para predicción
df_predict_v2['barrio_valido'] = df_predict_v2['l4_final'].apply(
    lambda x: x if x in barrios_validos else 'Otro'
)

# Encoders (usar los mismos entrenados)
try:
    df_predict_v2['ciudad_encoded'] = le_ciudad_v2.transform(df_predict_v2['l3_final'])
except:
    # Para ciudades no vistas, asignar código 0
    df_predict_v2['ciudad_encoded'] = 0

try:
    df_predict_v2['barrio_encoded'] = le_barrio.transform(df_predict_v2['barrio_valido'].fillna('Sin_Barrio'))
except:
    # Para barrios no vistos, asignar código 0
    df_predict_v2['barrio_encoded'] = 0

# Crear matriz X para predicción (misma estructura que entrenamiento)
X_pred_v2_features = []
for var in variables_v2:
    if var in df_predict_v2.columns:
        X_pred_v2_features.append(df_predict_v2[var].fillna(0))
    else:
        X_pred_v2_features.append(np.zeros(len(df_predict_v2)))

X_pred_v2 = pd.DataFrame(dict(zip(variables_v2, X_pred_v2_features)))

print(f"Features preparados: {X_pred_v2.shape}")
print(f"Variables: {variables_v2}")

# PASO 3: Realizar predicciones con modelo V2
print(f"\n🚀 REALIZANDO PREDICCIONES CON MODELO V2:")
print("-" * 45)

# Predecir con modelo optimizado
superficies_predichas_v2 = rf_model_v2.predict(X_pred_v2)

# Aplicar límites razonables basados en análisis
superficie_min = 20  # Mínimo realista
superficie_max = 300  # Máximo usado en entrenamiento
superficies_predichas_v2 = np.clip(superficies_predichas_v2, superficie_min, superficie_max)

print(f"✅ Predicciones realizadas: {len(superficies_predichas_v2):,}")
print(f"Superficie promedio predicha: {superficies_predichas_v2.mean():.1f} m²")
print(f"Superficie mediana predicha: {np.median(superficies_predichas_v2):.1f} m²")
print(f"Rango predicho: {superficies_predichas_v2.min():.1f} - {superficies_predichas_v2.max():.1f} m²")

# PASO 4: Integrar predicciones al dataset
print(f"\n📊 INTEGRANDO PREDICCIONES AL DATASET:")
print("-" * 45)

# Agregar predicciones V2 al dataset principal
df_clean['surface_ml_predicted'] = np.nan
df_clean.loc[mask_prediccion_v2, 'surface_ml_predicted'] = superficies_predichas_v2

# Crear columna final: original → extraída → ML predicted
df_clean['surface_final_v2'] = (
    df_clean['surface_total_final']
    .fillna(df_clean['surface_ml_predicted'])
)

# Estadísticas de completitud
superficie_antes_v2 = df_clean['surface_total_final'].notna().sum()
superficie_despues_v2 = df_clean['surface_final_v2'].notna().sum()
superficie_ml_completada = superficie_despues_v2 - superficie_antes_v2

print(f"Superficie antes (original + extraída): {superficie_antes_v2:,}")
print(f"Superficie después (+ ML): {superficie_despues_v2:,}")
print(f"✅ Completadas por ML: {superficie_ml_completada:,}")

cobertura_superficie_v2 = (superficie_despues_v2 / len(df_clean)) * 100
print(f"🎯 Cobertura total superficie: {cobertura_superficie_v2:.1f}%")

# Comparar con método anterior
if 'surface_predicted' in df_clean.columns:
    mejora_cobertura = cobertura_superficie_v2 - ((superficie_antes_v2 + df_clean['surface_predicted'].notna().sum()) / len(df_clean) * 100)
    print(f"📈 Mejora vs modelo V1: +{mejora_cobertura:.1f} puntos porcentuales")

print(f"\n🏆 FASE 11 COMPLETADA CON MODELO OPTIMIZADO")
print(f"💎 {superficie_ml_completada:,} superficies predichas con ML (R² = 0.807)")

🎯 APLICANDO MODELO V2 OPTIMIZADO
🔍 IDENTIFICANDO REGISTROS PARA PREDICCIÓN:
---------------------------------------------
Registros elegibles para predicción: 147,148

🔧 PREPARANDO FEATURES V2 PARA PREDICCIÓN:
---------------------------------------------
Features preparados: (147148, 9)
Variables: ['bedrooms_final', 'bathrooms_final', 'rooms_final', 'precio_relativo', 'densidad_habitaciones', 'ratio_banos_habitaciones', 'ciudad_encoded', 'barrio_encoded', 'tiene_barrio']

🚀 REALIZANDO PREDICCIONES CON MODELO V2:
---------------------------------------------
Registros elegibles para predicción: 147,148

🔧 PREPARANDO FEATURES V2 PARA PREDICCIÓN:
---------------------------------------------
Features preparados: (147148, 9)
Variables: ['bedrooms_final', 'bathrooms_final', 'rooms_final', 'precio_relativo', 'densidad_habitaciones', 'ratio_banos_habitaciones', 'ciudad_encoded', 'barrio_encoded', 'tiene_barrio']

🚀 REALIZANDO PREDICCIONES CON MODELO V2:
--------------------------------------

## 1️⃣2️⃣ **Fase 12: Validación Final y Resumen Ejecutivo** 🎯

**Objetivo:** Validar la calidad del dataset final y presentar resumen ejecutivo completo

**Componentes:**
1. **Validación de calidad** de todas las variables procesadas
2. **Métricas de completitud** antes vs después por cada fase
3. **Resumen ejecutivo** de innovaciones y logros
4. **Recomendaciones** para uso del dataset limpio

**Importancia:** Asegurar que el dataset está listo para modelos predictivos de precios inmobiliarios con la máxima calidad científica posible.

In [59]:
# ===============================================================
# FASE 12: VALIDACIÓN FINAL Y MÉTRICAS DE CALIDAD
# ===============================================================

print("🎯 FASE 12: VALIDACIÓN FINAL Y RESUMEN EJECUTIVO")
print("=" * 60)

# PASO 1: Métricas de calidad final del dataset
print("📊 MÉTRICAS DE CALIDAD FINAL DEL DATASET:")
print("-" * 50)

# Información básica del dataset final
total_registros = len(df_clean)
print(f"📋 Total registros finales: {total_registros:,}")

# Evaluación de variables críticas después del procesamiento
variables_finales = {
    'Precio': 'price',
    'Coordenadas (lat/lon)': ['lat_imputada', 'lon_imputada'], 
    'Ciudad': 'l3_final',
    'Barrio': 'l4_final',
    'Superficie': 'surface_final_v2',
    'Habitaciones': 'bedrooms_final',
    'Baños': 'bathrooms_final',
    'Rooms': 'rooms_final'
}

print(f"🎯 COMPLETITUD POR VARIABLE CRÍTICA:")
print("-" * 40)

for nombre, columna in variables_finales.items():
    if isinstance(columna, list):
        # Para coordenadas (ambas deben estar disponibles)
        disponible = df_clean[columna].notna().all(axis=1).sum()
    else:
        # Para variables simples
        if columna in df_clean.columns:
            disponible = df_clean[columna].notna().sum()
        else:
            disponible = 0
    
    porcentaje = (disponible / total_registros) * 100
    estado = "🟢 EXCELENTE" if porcentaje >= 80 else "🟡 BUENO" if porcentaje >= 60 else "🔴 MEJORAR"
    print(f"   {nombre:<20}: {disponible:>8,} ({porcentaje:>5.1f}%) {estado}")

# PASO 2: Distribución geográfica final
print(f"\n🌍 DISTRIBUCIÓN GEOGRÁFICA FINAL:")
print("-" * 35)

# Ciudades con más registros
ciudades_final = df_clean['l3_final'].value_counts().head(10)
print("Top 10 ciudades:")
for ciudad, count in ciudades_final.items():
    pct = (count / total_registros) * 100
    print(f"  {ciudad:<20}: {count:>6,} ({pct:>5.1f}%)")

# Barrios con más registros
print(f"\nBarrios con más datos:")
barrios_final = df_clean['l4_final'].value_counts().head(8)
for barrio, count in barrios_final.items():
    pct = (count / total_registros) * 100
    print(f"  {barrio:<20}: {count:>6,} ({pct:>5.1f}%)")

# PASO 3: Calidad de datos por origen
print(f"\n🔍 CALIDAD POR ORIGEN DE DATOS:")
print("-" * 35)

# Superficie por origen
superficie_original = df_clean['surface_total'].notna().sum()
superficie_extraida = df_clean['surface_extracted'].notna().sum() if 'surface_extracted' in df_clean.columns else 0
superficie_ml = df_clean['surface_ml_predicted'].notna().sum() if 'surface_ml_predicted' in df_clean.columns else 0

print(f"📐 SUPERFICIE:")
print(f"  Original: {superficie_original:,}")
print(f"  Text Mining: {superficie_extraida:,}")
print(f"  ML Predicted: {superficie_ml:,}")
print(f"  Total: {superficie_original + superficie_extraida + superficie_ml:,}")

# Coordenadas por origen
coords_originales = df_clean[['lat', 'lon']].notna().all(axis=1).sum()
coords_imputadas = (
    df_clean[['lat', 'lon']].isna().any(axis=1) & 
    df_clean[['lat_imputada', 'lon_imputada']].notna().all(axis=1)
).sum()

print(f"\n🗺️ COORDENADAS:")
print(f"  Originales: {coords_originales:,}")
print(f"  Imputadas: {coords_imputadas:,}")
print(f"  Total: {coords_originales + coords_imputadas:,}")

print(f"\n✅ VALIDACIÓN DE CALIDAD COMPLETADA")

🎯 FASE 12: VALIDACIÓN FINAL Y RESUMEN EJECUTIVO
📊 MÉTRICAS DE CALIDAD FINAL DEL DATASET:
--------------------------------------------------
📋 Total registros finales: 277,856
🎯 COMPLETITUD POR VARIABLE CRÍTICA:
----------------------------------------
   Precio              :  277,856 (100.0%) 🟢 EXCELENTE
   Coordenadas (lat/lon):  140,202 ( 50.5%) 🔴 MEJORAR
   Ciudad              :  275,508 ( 99.2%) 🟢 EXCELENTE
   Barrio              :   98,232 ( 35.4%) 🔴 MEJORAR
   Superficie          :  219,525 ( 79.0%) 🟡 BUENO
   Habitaciones        :  216,320 ( 77.9%) 🟡 BUENO
   Baños               :  265,233 ( 95.5%) 🟢 EXCELENTE
   Rooms               :  124,037 ( 44.6%) 🔴 MEJORAR

🌍 DISTRIBUCIÓN GEOGRÁFICA FINAL:
-----------------------------------
Top 10 ciudades:
  Medellín            : 219,839 ( 79.1%)
  Envigado            : 21,049 (  7.6%)
  Sabaneta            : 10,060 (  3.6%)
  Bello               :  7,915 (  2.8%)
  Itagui              :  6,186 (  2.2%)
  Rionegro            :  5,979 ( 

In [60]:
# ===============================================================
# RESUMEN EJECUTIVO: INNOVACIONES Y LOGROS CLAVE
# ===============================================================

print("🏆 RESUMEN EJECUTIVO: HABIDATA CLEANING PROJECT")
print("=" * 60)

# Transformación del dataset
dataset_inicial = 1_000_000  # Aproximado del dataset original colombiano
dataset_antioquia = len(df_clean)

print(f"📊 TRANSFORMACIÓN DEL DATASET:")
print("-" * 35)
print(f"Dataset inicial Colombia: ~{dataset_inicial:,} propiedades")
print(f"Dataset final Antioquia: {dataset_antioquia:,} propiedades")
print(f"Enfoque regional: {(dataset_antioquia/dataset_inicial*100):.1f}% conservado")

# INNOVACIONES CLAVE IMPLEMENTADAS
print(f"\n🚀 INNOVACIONES CLAVE DESARROLLADAS:")
print("-" * 40)

innovaciones = [
    {
        'nombre': 'Text Mining Dual-Source',
        'descripcion': 'Extracción de surface, bedrooms, bathrooms desde description + title',
        'impacto': f'{superficie_extraida:,} superficies + 316,377 features recuperados'
    },
    {
        'nombre': 'ML Predictivo Optimizado', 
        'descripcion': 'Random Forest con feature engineering avanzado (R² = 0.807)',
        'impacto': f'{superficie_ml:,} superficies predichas con alta precisión'
    },
    {
        'nombre': 'Imputación Geográfica',
        'descripcion': 'Coordenadas por centroides de barrio',
        'impacto': f'{coords_imputadas:,} coordenadas imputadas espacialmente'
    },
    {
        'nombre': 'Location Recovery',
        'descripcion': 'Extracción de ciudades y barrios desde texto libre',
        'impacto': 'Recovery de 33,593+ ubicaciones desde descriptions y titles'
    }
]

for i, innovacion in enumerate(innovaciones, 1):
    print(f"\n{i}. 💡 {innovacion['nombre'].upper()}")
    print(f"   🔧 {innovacion['descripcion']}")
    print(f"   📈 {innovacion['impacto']}")

# MÉTRICAS DE ÉXITO CLAVE
print(f"\n📈 MÉTRICAS DE ÉXITO CLAVE:")
print("-" * 30)

mejoras_criticas = [
    ('Superficie', '4,762 → 219,525', '+4,507% incremento'),
    ('Coordenadas', '114,582 → 140,202', '+22.4% incremento'), 
    ('Ciudades', 'N/A → 275,508', '99.2% completitud'),
    ('Baños', 'N/A → 265,233', '95.5% completitud'),
    ('Dataset Quality', 'Raw → Production-Ready', 'Listo para ML')
]

for variable, transformacion, mejora in mejoras_criticas:
    print(f"🎯 {variable:<12}: {transformacion:<20} ({mejora})")

# PREPARACIÓN PARA MODELOS ML
print(f"\n🤖 PREPARACIÓN PARA MODELOS PREDICTIVOS:")
print("-" * 45)

# Registros listos para ML de precios
registros_ml_ready = df_clean[
    df_clean['price'].notna() &
    df_clean['surface_final_v2'].notna() &
    df_clean['bedrooms_final'].notna() &
    df_clean['bathrooms_final'].notna() &
    df_clean['l3_final'].notna()
].shape[0]

print(f"📊 Registros listos para ML: {registros_ml_ready:,}")
print(f"📊 Cobertura ML-ready: {(registros_ml_ready/dataset_antioquia*100):.1f}%")

# Variables disponibles para modelos
variables_disponibles = [
    'price', 'surface_final_v2', 'bedrooms_final', 'bathrooms_final', 
    'rooms_final', 'l3_final', 'l4_final', 'lat_imputada', 'lon_imputada'
]

print(f"\n📋 Variables clave disponibles: {len(variables_disponibles)}")
for var in variables_disponibles:
    if var in df_clean.columns:
        completitud = (df_clean[var].notna().sum() / dataset_antioquia * 100)
        print(f"   • {var:<20}: {completitud:>5.1f}% completitud")

print(f"\n🎯 DATASET OPTIMIZADO PARA PREDICCIÓN DE PRECIOS INMOBILIARIOS")
print(f"✅ Calidad científica y reproducibilidad garantizada")

🏆 RESUMEN EJECUTIVO: HABIDATA CLEANING PROJECT
📊 TRANSFORMACIÓN DEL DATASET:
-----------------------------------
Dataset inicial Colombia: ~1,000,000 propiedades
Dataset final Antioquia: 277,856 propiedades
Enfoque regional: 27.8% conservado

🚀 INNOVACIONES CLAVE DESARROLLADAS:
----------------------------------------

1. 💡 TEXT MINING DUAL-SOURCE
   🔧 Extracción de surface, bedrooms, bathrooms desde description + title
   📈 67,615 superficies + 316,377 features recuperados

2. 💡 ML PREDICTIVO OPTIMIZADO
   🔧 Random Forest con feature engineering avanzado (R² = 0.807)
   📈 147,148 superficies predichas con alta precisión

3. 💡 IMPUTACIÓN GEOGRÁFICA
   🔧 Coordenadas por centroides de barrio
   📈 25,620 coordenadas imputadas espacialmente

4. 💡 LOCATION RECOVERY
   🔧 Extracción de ciudades y barrios desde texto libre
   📈 Recovery de 33,593+ ubicaciones desde descriptions y titles

📈 MÉTRICAS DE ÉXITO CLAVE:
------------------------------
🎯 Superficie  : 4,762 → 219,525      (+4,507% inc

In [61]:
# ===============================================================
# RECOMENDACIONES FINALES Y PRÓXIMOS PASOS
# ===============================================================

print("📋 RECOMENDACIONES FINALES Y PRÓXIMOS PASOS")
print("=" * 55)

print("🎯 PARA MODELOS DE PREDICCIÓN DE PRECIOS:")
print("-" * 45)

recomendaciones_ml = [
    "✅ Variables núcleo: price, surface_final_v2, bedrooms_final, bathrooms_final",
    "✅ Features geográficos: l3_final (ciudad), lat_imputada, lon_imputada",
    "✅ Filtrar registros ML-ready (202,805 disponibles - 73.0% del dataset)",
    "✅ Usar feature engineering implementado (ratios, densidades, precios relativos)",
    "⚠️ Validar coordenadas imputadas para análisis espacial crítico",
    "⚠️ Considerar rooms_final como feature opcional (44.6% completitud)"
]

for rec in recomendaciones_ml:
    print(f"   {rec}")

print(f"\n🔬 CALIDAD CIENTÍFICA GARANTIZADA:")
print("-" * 35)

calidad_cientifica = [
    "📝 Documentación completa de cada decisión metodológica",
    "🔄 Proceso reproducible con 12 fases bien definidas",
    "📊 Validación estadística de modelos ML (R² = 0.807)",
    "🧹 Limpieza sistemática de outliers y datos inconsistentes",
    "⚙️ Feature engineering justificado técnicamente",
    "🎯 Métricas de calidad monitoreadas en cada fase"
]

for criterio in calidad_cientifica:
    print(f"   {criterio}")

print(f"\n🚀 PRÓXIMOS PASOS SUGERIDOS:")
print("-" * 30)

proximos_pasos = [
    {
        'paso': 'Modelado Predictivo',
        'descripcion': 'Implementar modelos de predicción de precios usando dataset limpio',
        'prioridad': 'ALTA'
    },
    {
        'paso': 'Análisis Geoespacial',
        'descripcion': 'Explorar patrones de precios por zonas usando coordenadas',
        'prioridad': 'MEDIA'
    },
    {
        'paso': 'Expansión de Cobertura',
        'descripcion': 'Aplicar metodología a otros departamentos colombianos',
        'prioridad': 'BAJA'
    },
    {
        'paso': 'Automatización',
        'descripcion': 'Crear pipeline automatizado para datos inmobiliarios nuevos',
        'prioridad': 'MEDIA'
    }
]

for i, paso in enumerate(proximos_pasos, 1):
    prioridad_color = "🔴" if paso['prioridad'] == 'ALTA' else "🟡" if paso['prioridad'] == 'MEDIA' else "🟢"
    print(f"\n{i}. {prioridad_color} {paso['paso'].upper()} ({paso['prioridad']})")
    print(f"   {paso['descripcion']}")

print(f"\n" + "="*60)
print(f"🎉 PROYECTO HABIDATA COMPLETADO EXITOSAMENTE")
print(f"💎 Dataset de {dataset_antioquia:,} propiedades listo para análisis avanzado")
print(f"🏆 Estándar universitario de calidad científica alcanzado")
print(f"🚀 Listo para modelos de Machine Learning de predicción de precios")
print("="*60)

📋 RECOMENDACIONES FINALES Y PRÓXIMOS PASOS
🎯 PARA MODELOS DE PREDICCIÓN DE PRECIOS:
---------------------------------------------
   ✅ Variables núcleo: price, surface_final_v2, bedrooms_final, bathrooms_final
   ✅ Features geográficos: l3_final (ciudad), lat_imputada, lon_imputada
   ✅ Filtrar registros ML-ready (202,805 disponibles - 73.0% del dataset)
   ✅ Usar feature engineering implementado (ratios, densidades, precios relativos)
   ⚠️ Validar coordenadas imputadas para análisis espacial crítico
   ⚠️ Considerar rooms_final como feature opcional (44.6% completitud)

🔬 CALIDAD CIENTÍFICA GARANTIZADA:
-----------------------------------
   📝 Documentación completa de cada decisión metodológica
   🔄 Proceso reproducible con 12 fases bien definidas
   📊 Validación estadística de modelos ML (R² = 0.807)
   🧹 Limpieza sistemática de outliers y datos inconsistentes
   ⚙️ Feature engineering justificado técnicamente
   🎯 Métricas de calidad monitoreadas en cada fase

🚀 PRÓXIMOS PASOS SUG

## 📊 **Análisis Exhaustivo de Variables Faltantes - Dataset Final**

**Objetivo:** Evaluar completitud de todas las variables del dataset final después del procesamiento completo de 12 fases

**Componentes:**
1. **Análisis global** de valores faltantes por variable
2. **Impacto por tipo** de variable (críticas vs auxiliares)
3. **Patrones de missingness** y dependencias entre variables
4. **Estrategias de optimización** adicional si es necesaria

In [62]:
# ===============================================================
# ANÁLISIS EXHAUSTIVO DE VARIABLES FALTANTES - DATASET FINAL
# ===============================================================

print("🔍 ANÁLISIS EXHAUSTIVO DE VARIABLES FALTANTES")
print("=" * 60)

# PASO 1: Inventario completo de variables en el dataset
print("📋 INVENTARIO COMPLETO DE VARIABLES:")
print("-" * 40)

total_registros = len(df_clean)
print(f"Total registros en dataset final: {total_registros:,}")

# Obtener todas las columnas del dataset
todas_las_columnas = df_clean.columns.tolist()
print(f"Total variables en dataset: {len(todas_las_columnas)}")

# Categorizar variables por tipo/propósito
categorias_variables = {
    'Variables Críticas': [
        'price', 'surface_final_v2', 'bedrooms_final', 'bathrooms_final', 
        'l3_final', 'lat_imputada', 'lon_imputada'
    ],
    'Variables Geográficas': [
        'l2', 'l3', 'l3_final', 'l4', 'l4_final', 'lat', 'lon', 
        'lat_imputada', 'lon_imputada'
    ],
    'Variables de Propiedades': [
        'surface_total', 'surface_final_v2', 'bedrooms', 'bedrooms_final',
        'bathrooms', 'bathrooms_final', 'rooms', 'rooms_final', 'property_type'
    ],
    'Variables Textuales': [
        'title', 'description'
    ],
    'Variables Extraídas (Text Mining)': [
        'surface_extracted', 'bedrooms_extracted', 'bathrooms_extracted', 
        'rooms_extracted', 'l3_extracted', 'l4_extracted', 
        'l3_title_extracted', 'l4_title_extracted'
    ],
    'Variables ML/Predichas': [
        'surface_ml_predicted'
    ]
}

# PASO 2: Análisis de completitud por categoría
print(f"\n📊 ANÁLISIS DE COMPLETITUD POR CATEGORÍA:")
print("-" * 50)

for categoria, variables in categorias_variables.items():
    print(f"\n🏷️ {categoria.upper()}:")
    print("-" * (len(categoria) + 5))
    
    variables_existentes = [var for var in variables if var in df_clean.columns]
    
    if variables_existentes:
        for var in variables_existentes:
            no_nulos = df_clean[var].notna().sum()
            nulos = df_clean[var].isna().sum()
            porcentaje = (no_nulos / total_registros) * 100
            
            # Clasificar por nivel de completitud
            if porcentaje >= 95:
                estado = "🟢 EXCELENTE"
            elif porcentaje >= 80:
                estado = "🟡 BUENO"
            elif porcentaje >= 50:
                estado = "🟠 REGULAR"
            else:
                estado = "🔴 CRÍTICO"
            
            print(f"   {var:<25}: {no_nulos:>8,} ({porcentaje:>5.1f}%) | Faltantes: {nulos:>8,} {estado}")
    else:
        print("   ⚠️ No hay variables de esta categoría en el dataset")

print(f"\n✅ INVENTARIO COMPLETADO")

🔍 ANÁLISIS EXHAUSTIVO DE VARIABLES FALTANTES
📋 INVENTARIO COMPLETO DE VARIABLES:
----------------------------------------
Total registros en dataset final: 277,856
Total variables en dataset: 43

📊 ANÁLISIS DE COMPLETITUD POR CATEGORÍA:
--------------------------------------------------

🏷️ VARIABLES CRÍTICAS:
-----------------------
   price                    :  277,856 (100.0%) | Faltantes:        0 🟢 EXCELENTE
   surface_final_v2         :  219,525 ( 79.0%) | Faltantes:   58,331 🟠 REGULAR
   bedrooms_final           :  216,320 ( 77.9%) | Faltantes:   61,536 🟠 REGULAR
   bathrooms_final          :  265,233 ( 95.5%) | Faltantes:   12,623 🟢 EXCELENTE
   l3_final                 :  275,508 ( 99.2%) | Faltantes:    2,348 🟢 EXCELENTE
   lat_imputada             :  140,202 ( 50.5%) | Faltantes:  137,654 🟠 REGULAR
   lon_imputada             :  140,202 ( 50.5%) | Faltantes:  137,654 🟠 REGULAR

🏷️ VARIABLES GEOGRÁFICAS:
--------------------------
   l2                       :  277,856 (100.

In [63]:
# ===============================================================
# ANÁLISIS DE PATRONES DE MISSINGNESS Y DEPENDENCIAS
# ===============================================================

print("🕸️ ANÁLISIS DE PATRONES DE MISSINGNESS")
print("=" * 50)

# PASO 3: Top variables con más valores faltantes
print("🔻 TOP 15 VARIABLES CON MÁS VALORES FALTANTES:")
print("-" * 55)

# Calcular missingness para todas las variables
missingness_global = {}
for col in df_clean.columns:
    nulos = df_clean[col].isna().sum()
    porcentaje = (nulos / total_registros) * 100
    if nulos > 0:  # Solo variables con valores faltantes
        missingness_global[col] = {'count': nulos, 'percentage': porcentaje}

# Ordenar por cantidad de faltantes
sorted_missing = sorted(missingness_global.items(), key=lambda x: x[1]['count'], reverse=True)

print(f"{'#':<3} {'Variable':<30} {'Faltantes':<12} {'%':<8} {'Estado'}")
print("-" * 70)

for i, (var, stats) in enumerate(sorted_missing[:15], 1):
    count = stats['count']
    pct = stats['percentage']
    
    if pct >= 70:
        estado = "🔴 CRÍTICO"
    elif pct >= 50:
        estado = "🟠 ALTO"
    elif pct >= 20:
        estado = "🟡 MEDIO"
    else:
        estado = "🟢 BAJO"
    
    print(f"{i:>2d}. {var:<30} {count:>8,} {pct:>6.1f}% {estado}")

# PASO 4: Variables críticas para ML - análisis específico
print(f"\n🎯 ANÁLISIS ESPECÍFICO PARA ML:")
print("-" * 35)

variables_ml_criticas = [
    'price', 'surface_final_v2', 'bedrooms_final', 'bathrooms_final', 
    'l3_final', 'lat_imputada', 'lon_imputada'
]

print("Variables críticas para modelos predictivos:")
registros_completos_ml = 0

# Crear máscara para registros completos en variables críticas
mask_completo = pd.Series([True] * len(df_clean))

for var in variables_ml_criticas:
    if var in df_clean.columns:
        no_nulos = df_clean[var].notna().sum()
        porcentaje = (no_nulos / total_registros) * 100
        
        # Actualizar máscara
        mask_completo = mask_completo & df_clean[var].notna()
        
        status = "✅" if porcentaje >= 80 else "⚠️" if porcentaje >= 60 else "❌"
        print(f"   {status} {var:<20}: {no_nulos:>8,} ({porcentaje:>5.1f}%)")

registros_completos_ml = mask_completo.sum()
print(f"\n🎯 Registros completos para ML: {registros_completos_ml:,} ({(registros_completos_ml/total_registros*100):.1f}%)")

# PASO 5: Combinaciones problemáticas
print(f"\n⚠️ COMBINACIONES PROBLEMÁTICAS:")
print("-" * 35)

# Sin superficie Y sin coordenadas
sin_superficie_ni_coords = (
    df_clean['surface_final_v2'].isna() & 
    df_clean[['lat_imputada', 'lon_imputada']].isna().any(axis=1)
).sum()

# Sin habitaciones Y sin superficie
sin_habitaciones_ni_superficie = (
    df_clean['bedrooms_final'].isna() & 
    df_clean['surface_final_v2'].isna()
).sum()

# Sin ubicación específica (ni ciudad ni barrio)
sin_ubicacion = (
    df_clean['l3_final'].isna() & 
    df_clean['l4_final'].isna()
).sum()

print(f"Sin superficie NI coordenadas: {sin_superficie_ni_coords:,}")
print(f"Sin habitaciones NI superficie: {sin_habitaciones_ni_superficie:,}")
print(f"Sin ubicación específica: {sin_ubicacion:,}")

print(f"\n✅ ANÁLISIS DE PATRONES COMPLETADO")

🕸️ ANÁLISIS DE PATRONES DE MISSINGNESS
🔻 TOP 15 VARIABLES CON MÁS VALORES FALTANTES:
-------------------------------------------------------
#   Variable                       Faltantes    %        Estado
----------------------------------------------------------------------
 1. l5                              277,856  100.0% 🔴 CRÍTICO
 2. l6                              277,856  100.0% 🔴 CRÍTICO
 3. l3_title_extracted              277,534   99.9% 🔴 CRÍTICO
 4. l3_extracted                    276,837   99.6% 🔴 CRÍTICO
 5. surface_total                   273,094   98.3% 🔴 CRÍTICO
 6. surface_covered                 272,647   98.1% 🔴 CRÍTICO
 7. l4_title_extracted              271,379   97.7% 🔴 CRÍTICO
 8. bathrooms_extracted             265,032   95.4% 🔴 CRÍTICO
 9. price_period                    257,063   92.5% 🔴 CRÍTICO
10. l4_extracted                    252,081   90.7% 🔴 CRÍTICO
11. rooms                           244,787   88.1% 🔴 CRÍTICO
12. l4                              211,87